In [39]:
# Automated Path Configuration for Weekly Analysis
import os
import glob
import calendar
import pandas as pd

def get_month_year_combinations(latest_month_year):
    """
    Get the latest month, last month, and last year combinations
    """
    # Parse the latest month-year
    month_name, year = latest_month_year.split('-')
    year = int(year)
    month_num = list(calendar.month_name).index(month_name)
    
    # Calculate last month
    if month_num == 1:  # January
        last_month_num = 12
        last_month_year_num = year - 1
    else:
        last_month_num = month_num - 1
        last_month_year_num = year
    
    # Calculate last year same month
    last_year_month_num = month_num
    last_year_year_num = year - 1
    
    # Convert back to names
    last_month_name = calendar.month_name[last_month_num]
    last_year_month_name = calendar.month_name[last_year_month_num]
    
    return {
        'latest': {'month': month_name, 'year': year, 'folder': f"{month_name}-{year}"},
        'last_month': {'month': last_month_name, 'year': last_month_year_num, 'folder': f"{last_month_name}-{last_month_year_num}"},
        'last_year': {'month': last_year_month_name, 'year': last_year_year_num, 'folder': f"{last_year_month_name}-{last_year_year_num}"}
    }

def find_file_by_keyword(folder_path, keyword):
    """
    Find a file in the folder that contains the keyword in its name
    """
    if not os.path.exists(folder_path):
        return None
    
    files = os.listdir(folder_path)
    for file in files:
        if keyword.lower() in file.lower() and file.endswith('.xlsx'):
            return os.path.join(folder_path, file)
    return None

def get_sheet_name_with_keyword(file_path, keyword):
    """
    Get the sheet name that contains the keyword
    """
    try:
        xl = pd.ExcelFile(file_path)
        for sheet_name in xl.sheet_names:
            if keyword.lower() in sheet_name.lower():
                return sheet_name
        # If no sheet with keyword found, return the first sheet
        return xl.sheet_names[0] if xl.sheet_names else None
    except:
        return None

def setup_automated_paths_weekly(latest_month_year, dsr_folder_path=None):
    """
    Setup all paths automatically for weekly analysis
    
    Parameters:
    latest_month_year: str - Format: "June-2025"
    dsr_folder_path: str - Full path to DSR folder (e.g., "C:/Users/Username/Documents/DSR")
                           If None, defaults to "DSR" in current directory
    
    Returns:
    dict containing all the required paths and configurations
    """
    
    # Get month-year combinations
    dates = get_month_year_combinations(latest_month_year)
    
    # Base DSR folder path - use provided path or default to current directory
    if dsr_folder_path is None:
        dsr_path = os.path.join(os.getcwd(), "DSR")
    else:
        dsr_path = os.path.abspath(dsr_folder_path)
        
    print(f"🔍 Looking for DSR folder at: {dsr_path}")
    
    # Prepare results
    sheet_info = []
    sessions_info = []
    target_info = {}
    
    # Process each period (last_month, last_year, latest)
    periods = ['last_month', 'last_year', 'latest']
    display_names = [
        f"{dates['last_month']['month']} {dates['last_month']['year'] % 100}",  # May 25
        f"{dates['last_year']['month']} {dates['last_year']['year'] % 100}",   # June 24
        f"{dates['latest']['month']} {dates['latest']['year'] % 100}"          # June 25
    ]
    
    for i, period in enumerate(periods):
        period_data = dates[period]
        folder_path = os.path.join(dsr_path, period_data['folder'])
        
        # Find invoice file
        invoice_file = find_file_by_keyword(folder_path, 'invoice')
        if invoice_file:
            # Get the first sheet (since invoice files have only one sheet)
            try:
                xl = pd.ExcelFile(invoice_file)
                sheet_name = xl.sheet_names[0] if xl.sheet_names else 'Sheet1'
            except:
                sheet_name = 'Sheet1'
            
            # Make path relative to current working directory
            rel_path = os.path.relpath(invoice_file, os.getcwd())
            sheet_info.append((rel_path, sheet_name, display_names[i]))
        
        # Find traffic/session file
        traffic_file = find_file_by_keyword(folder_path, 'traffic')
        if traffic_file:
            download_sheet = get_sheet_name_with_keyword(traffic_file, 'download')
            if not download_sheet:
                # If no download sheet found, get the first sheet
                try:
                    xl = pd.ExcelFile(traffic_file)
                    download_sheet = xl.sheet_names[0] if xl.sheet_names else 'Sheet1'
                except:
                    download_sheet = 'Sheet1'
            
            # Make path relative to current working directory
            rel_path = os.path.relpath(traffic_file, os.getcwd())
            sessions_info.append((rel_path, download_sheet, display_names[i]))
    
    # Setup target information (using latest month)
    latest_folder = os.path.join(dsr_path, dates['latest']['folder'])
    target_file = find_file_by_keyword(latest_folder, 'target')
    if target_file:
        target_sheet = get_sheet_name_with_keyword(target_file, 'target')
        if not target_sheet:
            target_sheet = 'Target'  # Default if not found
        
        target_info = {
            'path': os.path.relpath(target_file, os.getcwd()),
            'sheet': target_sheet
        }
    
    return {
        'sheet_info': sheet_info,
        'sessions_info': sessions_info,
        'target_info': target_info,
        'dates': dates
    }

# Get user input for the latest month and DSR path
print("📊 Weekly Analysis - Automated Path Configuration")
print("=" * 50)
print("Please enter the latest month in format 'Month-Year' (e.g., 'June-2025'):")
print("You'll also be asked for the DSR folder path if it's not in the default location.")
print("Note: Make sure the DSR folder exists with the appropriate month folders.")
print("This will configure both invoice data and session data automatically.")

📊 Weekly Analysis - Automated Path Configuration
Please enter the latest month in format 'Month-Year' (e.g., 'June-2025'):
You'll also be asked for the DSR folder path if it's not in the default location.
Note: Make sure the DSR folder exists with the appropriate month folders.
This will configure both invoice data and session data automatically.


In [40]:
# Apply Automated Configuration
latest_month_year = input("Enter the latest month-year (e.g., 'June-2025'): ").strip()
dsr_folder_path = input("Enter full path to DSR folder (or press Enter for default './DSR'): ").strip() or None

# Setup all paths automatically
try:
    config = setup_automated_paths_weekly(latest_month_year, dsr_folder_path)
    
    # Extract configuration
    sheet_info = config['sheet_info']
    sessions_info = config['sessions_info']
    target_config = config['target_info']
    
    print(f"\n✅ Weekly Analysis Configuration successful!")
    if dsr_folder_path:
        print(f"📁 Using DSR folder: {dsr_folder_path}")
    print(f"📁 Found {len(sheet_info)} invoice files:")
    for i, (path, sheet, display) in enumerate(sheet_info):
        print(f"   {i+1}. {display}: {path} -> {sheet}")
    
    print(f"\n📊 Found {len(sessions_info)} session/traffic files:")
    for i, (path, sheet, display) in enumerate(sessions_info):
        print(f"   {i+1}. {display}: {path} -> {sheet}")
    
    if target_config:
        print(f"\n🎯 Target file: {target_config['path']} -> {target_config['sheet']}")
        TARGET_PATH = target_config['path']
        TARGET_SHEET = target_config['sheet']
    else:
        print("\n⚠️  No target file found - using fallback")
        TARGET_PATH = 'test2/Target_June_25.xlsx'
        TARGET_SHEET = 'Target-June25'
    
    print(f"\n🚀 Ready for weekly analysis with automated paths!")
    
except Exception as e:
    print(f"❌ Error in automated setup: {e}")
    print("🔄 Falling back to manual configuration...")
    
    # Fallback to manual configuration
    sheet_info = [
        ('test2/may25-final.xlsx', 'Sheet1', 'May 25'),   # Last month raw sheet
        ('test2/June24_Invoice.xlsx', 'Raw data June 24', 'June 24'),        # Last year raw sheet
        ('test2/June25.xlsx', 'Sheet1', 'June 25')                # Latest month raw sheet
    ]
    
    sessions_info = [
        ('test2/May_2025_Daily traffic (2).xlsx', 'download - 2025-01-08T160122.10', 'May 25'), # Last month session
        ('test2/June Traffic -2024.xlsx', 'download - 2025-06-03T09524 (2)', 'June 24'),  # Last year session
        ('test2/June_2025_Daily traffic.xlsx', 'download - 2025-01-08T160122.10', 'June 25')   # Current month session
    ]
    
    TARGET_PATH = 'test2/Target_June_25.xlsx'
    TARGET_SHEET = 'Target-June25'

🔍 Looking for DSR folder at: c:\Users\91843\Documents\VsCode Codes\ReportAutomation\DSR

✅ Weekly Analysis Configuration successful!
📁 Found 3 invoice files:
   1. May 25: DSR\May-2025\may25_Invoice.xlsx -> Sheet1
   2. June 24: DSR\June-2024\June24_Invoice.xlsx -> Raw data June 24
   3. June 25: DSR\June-2025\June25_Invoice.xlsx -> Sheet1

📊 Found 3 session/traffic files:
   1. May 25: DSR\May-2025\May_2025_Daily_traffic.xlsx -> download - 2025-01-08T160122.10
   2. June 24: DSR\June-2024\June_2024_Daily_traffic.xlsx -> download - 2025-06-03T095249.89
   3. June 25: DSR\June-2025\June_2025_Daily_traffic.xlsx -> download - 2025-01-08T160122.10

🎯 Target file: DSR\June-2025\Target_June_25.xlsx -> Target-June25

🚀 Ready for weekly analysis with automated paths!

✅ Weekly Analysis Configuration successful!
📁 Found 3 invoice files:
   1. May 25: DSR\May-2025\may25_Invoice.xlsx -> Sheet1
   2. June 24: DSR\June-2024\June24_Invoice.xlsx -> Raw data June 24
   3. June 25: DSR\June-2025\June25

In [41]:
# Get necessary imports
import pandas as pd
import openpyxl
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment, numbers
from openpyxl.utils import get_column_letter
from datetime import datetime
import calendar
from copy import copy  # For copying Excel cell styles

In [42]:
# Define target Excel file 
output_path = 'weekly.xlsx'

# ✅ USING AUTOMATED CONFIGURATION
# sheet_info, sessions_info, TARGET_PATH, and TARGET_SHEET are now configured automatically above
# If automation failed, they fall back to manual configuration

# Verify configuration is loaded
if 'sheet_info' not in globals():
    print("⚠️  sheet_info not found - please run the automated configuration cells above")
if 'sessions_info' not in globals():
    print("⚠️  sessions_info not found - please run the automated configuration cells above")

# Get latest invoice data for max day calculation
latest_path, latest_sheet, latest_display = sheet_info[-1]
latest_df = pd.read_excel(latest_path, sheet_name=latest_sheet)
max_invoice_day = pd.to_datetime(latest_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day.max()

print(f"📊 Configuration Summary:")
print(f"   Sheet Info: {len(sheet_info)} files configured")
print(f"   Sessions Info: {len(sessions_info)} files configured") 
print(f"   Target: {TARGET_PATH} -> {TARGET_SHEET}")
print(f"   Max Invoice Day: {max_invoice_day}")

# Create unique identifiers by combining file path and sheet name
LAST_MONTH_ID = f"{sheet_info[0][0]}_{sheet_info[0][1]}"
LAST_YEAR_ID = f"{sheet_info[1][0]}_{sheet_info[1][1]}"
CURRENT_ID = f"{sheet_info[2][0]}_{sheet_info[2][1]}"

# Dynamic constants extracted from sheet_info for easy reference
LAST_MONTH_PATH = sheet_info[0][0]      # File path for last month
LAST_MONTH_SHEET = sheet_info[0][1]     # Sheet name for last month
LAST_MONTH_DISPLAY = sheet_info[0][2]   # Display name for last month

LAST_YEAR_PATH = sheet_info[1][0]       # File path for last year
LAST_YEAR_SHEET = sheet_info[1][1]      # Sheet name for last year
LAST_YEAR_DISPLAY = sheet_info[1][2]    # Display name for last year

CURRENT_PATH = sheet_info[2][0]         # File path for current month
CURRENT_SHEET = sheet_info[2][1]        # Sheet name for current month
CURRENT_DISPLAY = sheet_info[2][2]      # Display name for current month

# Create dynamic periods list from sheet_info display names
periods = [LAST_MONTH_DISPLAY, LAST_YEAR_DISPLAY, CURRENT_DISPLAY]

print("📊 Dynamic Configuration Loaded:")
print("=" * 40)
print(f"Last Month:   {LAST_MONTH_DISPLAY} (ID: {LAST_MONTH_ID})")
print(f"Last Year:    {LAST_YEAR_DISPLAY} (ID: {LAST_YEAR_ID})")  
print(f"Current:      {CURRENT_DISPLAY} (ID: {CURRENT_ID})")
print(f"Periods:      {periods}")
print("=" * 40)

📊 Configuration Summary:
   Sheet Info: 3 files configured
   Sessions Info: 3 files configured
   Target: DSR\June-2025\Target_June_25.xlsx -> Target-June25
   Max Invoice Day: 12
📊 Dynamic Configuration Loaded:
Last Month:   May 25 (ID: DSR\May-2025\may25_Invoice.xlsx_Sheet1)
Last Year:    June 24 (ID: DSR\June-2024\June24_Invoice.xlsx_Raw data June 24)
Current:      June 25 (ID: DSR\June-2025\June25_Invoice.xlsx_Sheet1)
Periods:      ['May 25', 'June 24', 'June 25']


In [43]:
# Get target sums by day and channel using constants - WITH DEBUG OUTPUT
print("🔍 DEBUGGING TARGET DATA STRUCTURE")
print("=" * 50)

target_df = pd.read_excel(TARGET_PATH, sheet_name=TARGET_SHEET)
print(f"📊 Raw Target Data Shape: {target_df.shape}")
print(f"📋 Target Data Columns: {list(target_df.columns)}")
print("\n📅 First 10 rows of Target Data:")
print(target_df.head(10))

print("\n🏷️ Unique Channels in Target Data:")
print(target_df['Channel'].unique())

print("\n🏷️ Unique Categories in Target Data:")
if 'Category' in target_df.columns:
    print(target_df['Category'].unique())
else:
    print("No 'Category' column found in target data")

# Modified to include 'Category' in the grouping if it exists
if 'Category' in target_df.columns:
    target_sums = target_df.groupby(['Date', 'Channel', 'Category'])['Target'].sum().unstack(level=['Channel', 'Category'], fill_value=0).round(6)
else:
    target_sums = target_df.groupby(['Date', 'Channel'])['Target'].sum().unstack(level='Channel', fill_value=0).round(6)

print(f"\n📊 Processed Target Sums Shape: {target_sums.shape}")
print("\n📋 Target Sums Sample:")
print(target_sums.head())

print("=" * 50)

🔍 DEBUGGING TARGET DATA STRUCTURE
📊 Raw Target Data Shape: (806, 5)
📋 Target Data Columns: ['Date', 'Week', 'Channel', 'Category', 'Target']

📅 First 10 rows of Target Data:
   Date  Week   Channel Category      Target
0     1     1  Jumbo.ae      ACC  2100.00000
1     2     2  Jumbo.ae      ACC  2160.00000
2     3     2  Jumbo.ae      ACC  2125.16129
3     4     2  Jumbo.ae      ACC  2125.16129
4     5     2  Jumbo.ae      ACC  2220.00000
5     6     2  Jumbo.ae      ACC  1980.00000
6     7     2  Jumbo.ae      ACC  1980.00000
7     8     2  Jumbo.ae      ACC  2100.00000
8     9     3  Jumbo.ae      ACC  2040.00000
9    10     3  Jumbo.ae      ACC  1980.00000

🏷️ Unique Channels in Target Data:
['Jumbo.ae' 'EA']

🏷️ Unique Categories in Target Data:
['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']

📊 Processed Target Sums Shape: (31, 26)

📋 Target Sums Sample:
Channel           EA                                                       \
Category         ACC   

In [44]:
# 📅 Week Analysis Configuration - User Input Based
from datetime import datetime
import calendar

print("🗓️  WEEK ANALYSIS CONFIGURATION")
print("=" * 50)
print("📝 Choose your preferred starting day for weekly analysis:")
print("   Options: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday")
print()

# Get the first day of the month from user
while True:
    first_day = input('Enter the first day of the month (Monday/Tuesday/Wednesday/Thursday/Friday/Saturday/Sunday): ').strip().lower()
    if first_day in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
        break
    print('Invalid input! Please enter a valid day name.')

# Create a mapping of days to their position in a week (0=Monday to 6=Sunday)
day_positions = {
    'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
    'friday': 4, 'saturday': 5, 'sunday': 6
}
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Get position of the first day (0-6, where 0 is Monday)
first_day_position = day_positions[first_day]
start_day = day_names[first_day_position].capitalize()

print(f"✅ First day of month set to: {start_day}")
print(f"📊 Configuration: Week analysis starts based on {start_day} as first day")
print("=" * 50)

def get_week_info(day_of_month, first_day_pos):
    """Get week information for a given day.
    Returns (week_number, is_first_partial_week)"""
    # For days in the first partial week
    if first_day_pos > 0:  # If month doesn't start on Monday
        days_till_next_monday = 7 - first_day_pos
        if day_of_month <= days_till_next_monday:
            return 1, True
        # Adjust day number to calculate remaining weeks
        adjusted_day = day_of_month - days_till_next_monday
        return (adjusted_day - 1) // 7 + 2, False
    else:  # If month starts on Monday
        return (day_of_month - 1) // 7 + 1, False

def get_week_number(day_of_month, first_day_pos):
    """Get week number for a given day of month"""
    week_num, _ = get_week_info(day_of_month, first_day_pos)
    return week_num

def get_week_label(week_num):
    """Get descriptive week label"""
    return f"Week {week_num}"

def get_day_name(day_number, first_day_pos):
    """Get the day name for a given day of month"""
    # Calculate the day of week (0-6, where 0 is Monday)
    day_of_week = (first_day_pos + day_number - 1) % 7
    return day_names[day_of_week]

# Test the calculation
print(f"\n📊 Week Calculation Test (First day: {start_day})")
print("=" * 50)
for day in range(1, 16):  # Show first 15 days
    week_num = get_week_number(day, first_day_position)
    week_label = get_week_label(week_num)
    day_name = get_day_name(day, first_day_position)
    print(f"Day {day:2d} ({day_name:9s}) → {week_label}")

# Calculate total weeks for a typical 31-day month
max_weeks = max([get_week_number(day, first_day_position) for day in range(1, 32)])
print(f"\n📈 Total weeks in a 31-day month: {max_weeks}")
print(f"📋 Analysis will group days based on {start_day} as first day of month")
print("=" * 50)

# Store variables for use in other cells
start_day_num = first_day_position  # For compatibility with existing code


🗓️  WEEK ANALYSIS CONFIGURATION
📝 Choose your preferred starting day for weekly analysis:
   Options: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday

✅ First day of month set to: Sunday
📊 Configuration: Week analysis starts based on Sunday as first day

📊 Week Calculation Test (First day: Sunday)
Day  1 (Sunday   ) → Week 1
Day  2 (Monday   ) → Week 2
Day  3 (Tuesday  ) → Week 2
Day  4 (Wednesday) → Week 2
Day  5 (Thursday ) → Week 2
Day  6 (Friday   ) → Week 2
Day  7 (Saturday ) → Week 2
Day  8 (Sunday   ) → Week 2
Day  9 (Monday   ) → Week 3
Day 10 (Tuesday  ) → Week 3
Day 11 (Wednesday) → Week 3
Day 12 (Thursday ) → Week 3
Day 13 (Friday   ) → Week 3
Day 14 (Saturday ) → Week 3
Day 15 (Sunday   ) → Week 3

📈 Total weeks in a 31-day month: 6
📋 Analysis will group days based on Sunday as first day of month
✅ First day of month set to: Sunday
📊 Configuration: Week analysis starts based on Sunday as first day

📊 Week Calculation Test (First day: Sunday)
Day  1 (Sunday   

In [45]:
# Collect week-wise and TYPE-wise sums for each sheet using simplified week calculation
results = []
type_results = []
idg_results = []  # New list for IDG results

for path, sheet, display_name in sheet_info:
    df = pd.read_excel(path, sheet_name=sheet)
    filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
    filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day


    original_len = len(filtered_df)
    filtered_df = filtered_df[filtered_df['InvoiceDay'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_df)} rows with InvoiceDay > {max_invoice_day}")


    
    # Add week number calculation using the simplified function
    filtered_df['WeekNumber'] = filtered_df['InvoiceDay'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    # Map CC to Jumbo.ae in the TYPE column
    filtered_df['TYPE'] = filtered_df['TYPE'].replace('CC', 'Jumbo.ae')
    
    # Week-wise sum - use a unique identifier combining file path and sheet name
    unique_id = f"{path}_{sheet}"
    invoice_week_sum = filtered_df.groupby('WeekNumber')['Amount Invoiced W.O. VAT'].sum()
    results.append((unique_id, invoice_week_sum, display_name))
    
    # TYPE-wise sum for Jumbo.ae and EA by week - use unique identifier
    filtered_type = filtered_df[filtered_df['TYPE'].isin(['Jumbo.ae', 'EA'])]
    sum_by_week_type = filtered_type.groupby(['WeekNumber', 'TYPE'])['Amount Invoiced W.O. VAT'].sum().unstack(fill_value=0)
    type_results.append((unique_id, sum_by_week_type, display_name))
    
    # IDG-wise sum by week (new addition) - use unique identifier
    sum_by_week_idg = filtered_df.pivot_table(
        values='Amount Invoiced W.O. VAT',
        index='WeekNumber',
        columns='idg',
        aggfunc='sum',
        fill_value=0
    )
    idg_results.append((unique_id, display_name, sum_by_week_idg))

type_results

# Target Data Processing Functions with Channel Filtering and Debug Output
print("🎯 TARGET DATA PROCESSING WITH CHANNEL FILTERING")
print("=" * 60)

def process_target_data_by_week_and_channel(channel_filter=None):
    """
    Process target data and group by week and IDG (Category), with optional channel filtering
    
    Args:
        channel_filter: str, optional - Filter by 'Jumbo.ae' or 'EA' or None for all
    
    Returns:
        DataFrame with weeks as index and IDG (Category) as columns
    """
    print(f"\n🔄 Processing target data for channel: {channel_filter or 'ALL CHANNELS'}")
    
    # Read target data
    target_df = pd.read_excel(TARGET_PATH, sheet_name=TARGET_SHEET)
    
    print(f"📊 Original target data shape: {target_df.shape}")
    
    # Apply channel filter if specified
    if channel_filter:
        filtered_target = target_df[target_df['Channel'] == channel_filter].copy()
        print(f"🔍 After {channel_filter} filter: {filtered_target.shape}")
        
        if filtered_target.empty:
            print(f"⚠️ WARNING: No target data found for channel '{channel_filter}'")
            print(f"📅 Available channels: {target_df['Channel'].unique()}")
            return pd.DataFrame()
    else:
        filtered_target = target_df.copy()
    
    # Use the 'Date' column directly as 'Day' as it contains day numbers
    filtered_target['Day'] = filtered_target['Date']
    
    # Ensure 'Day' column is numeric and handle potential errors (e.g., non-numeric values)
    filtered_target['Day'] = pd.to_numeric(filtered_target['Day'], errors='coerce')

    # Remove rows with invalid dates (where 'Day' could not be converted to numeric)
    before_date_filter = len(filtered_target)
    filtered_target = filtered_target.dropna(subset=['Day'])
    after_date_filter = len(filtered_target)
    
    if before_date_filter != after_date_filter:
        print(f"⚠️ Removed {before_date_filter - after_date_filter} rows with invalid 'Day' values (non-numeric or empty).")
    
    # Convert 'Day' to integer type after cleaning
    if not filtered_target.empty:
        filtered_target['Day'] = filtered_target['Day'].astype(int)

    original_len = len(filtered_target)
    filtered_target = filtered_target[filtered_target['Day'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_target)} rows with Day > {max_invoice_day}")
    # Add week number calculation
    filtered_target['WeekNumber'] = filtered_target['Day'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    print(f"📅 Date range in target: {filtered_target['Day'].min()} to {filtered_target['Day'].max()}")
    print(f"📋 Week range: {filtered_target['WeekNumber'].min()} to {filtered_target['WeekNumber'].max()}")
    
    # Check if Category column exists and use it, otherwise use Channel as IDG
    if 'Category' in filtered_target.columns:
        print(f"🏷️ Using 'Category' column for IDG grouping")
        print(f"📅 Available categories: {filtered_target['Category'].unique()}")
        
        # Group by Week and Category to get target by week and IDG
        target_by_week = filtered_target.groupby(['WeekNumber', 'Category'])['Target'].sum().unstack(level='Category', fill_value=0)
    else:
        print(f"🏷️ No 'Category' column found, using 'Channel' for IDG grouping")
        
        # Group by Week and Channel to get target by week
        target_by_week = filtered_target.groupby(['WeekNumber', 'Channel'])['Target'].sum().unstack(level='Channel', fill_value=0)
    
    # Clean column names
    target_by_week.columns.name = None
    
    print(f"📊 Final target pivot shape: {target_by_week.shape}")
    print(f"📋 Target IDGs/Categories: {list(target_by_week.columns)}")
    
    return target_by_week

# Test target processing for all channels
print("\n🔬 TESTING TARGET DATA PROCESSING:")
print("-" * 40)

# Import IPython display to avoid conflicts with overridden display variable
from IPython.display import display

# Test 1: All channels
all_channels_target = process_target_data_by_week_and_channel()
if not all_channels_target.empty:
    print("\n📋 ALL CHANNELS TARGET DATA:")
    display(all_channels_target.round(2))

# Test 2: Jumbo.ae only  
jumbo_target = process_target_data_by_week_and_channel('Jumbo.ae')
if not jumbo_target.empty:
    print("\n📋 JUMBO.AE TARGET DATA:")
    display(jumbo_target.round(2))

# Test 3: EA only
ea_target = process_target_data_by_week_and_channel('EA')
if not ea_target.empty:
    print("\n📋 EA TARGET DATA:")
    display(ea_target.round(2))

print("\n=" * 60)

📉 May 25: Filtered 3217 rows with InvoiceDay > 12
📉 June 24: Filtered 2307 rows with InvoiceDay > 12
📉 June 24: Filtered 2307 rows with InvoiceDay > 12
📉 June 25: Filtered 0 rows with InvoiceDay > 12
🎯 TARGET DATA PROCESSING WITH CHANNEL FILTERING

🔬 TESTING TARGET DATA PROCESSING:
----------------------------------------

🔄 Processing target data for channel: ALL CHANNELS
📊 Original target data shape: (806, 5)
📉 June 25: Filtered 494 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']

📋 ALL CHANNELS TARGET DATA:
📉 June 25: Filtered 0 rows with InvoiceDay > 12
🎯 TARGET DATA PROCESSING WITH CHANNEL FILTERING

🔬 TESTING TARGET DATA PROCESSING:
-------------------------------------

,ACC,AV,GAM,IMG,IT,LA,MM,OAD,PG,S&N,SDA,TEL,WTC
WeekNumber,,,,,,,,,,,,,
1,2548.43,11651.67,7218.74,5460.00,48381.92,5692.00,4242.75,5306.09,4759.58,6288.45,7997.61,117195.09,3888.29
2,17705.20,80813.22,50009.52,37851.77,331591.59,39602.13,29477.49,36626.43,32570.98,43495.99,55102.99,803135.85,26903.79
3,9390.12,42651.07,26304.83,19950.32,168911.63,21090.71,15635.25,19061.56,16514.91,22773.22,28519.89,408997.62,14100.26



🔄 Processing target data for channel: Jumbo.ae
📊 Original target data shape: (806, 5)
🔍 After Jumbo.ae filter: (403, 5)
📉 June 25: Filtered 247 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']

📋 JUMBO.AE TARGET DATA:


,ACC,AV,GAM,IMG,IT,LA,MM,OAD,PG,S&N,SDA,TEL,WTC
WeekNumber,,,,,,,,,,,,,
1,2100.00,9100.00,5425.0,4200.00,23190.67,4900.00,3500.00,3500.00,2100.00,4473.43,4900.00,55897.49,2800.00
2,14690.32,63658.06,37950.0,29380.65,162227.82,34277.42,24483.87,24483.87,14690.32,31293.38,34277.42,391024.85,19587.10
3,7945.16,34429.03,20525.0,15890.32,87739.82,18538.71,13241.94,13241.94,7945.16,16924.81,18538.71,211483.14,10593.55



🔄 Processing target data for channel: EA
📊 Original target data shape: (806, 5)
🔍 After EA filter: (403, 5)
📉 June 25: Filtered 247 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']

📋 EA TARGET DATA:


,ACC,AV,GAM,IMG,IT,LA,MM,OAD,PG,S&N,SDA,TEL,WTC
WeekNumber,,,,,,,,,,,,,
1,448.43,2551.67,1793.74,1260.00,25191.25,792.00,742.75,1806.09,2659.58,1815.02,3097.61,61297.60,1088.29
2,3014.88,17155.15,12059.52,8471.13,169363.76,5324.71,4993.62,12142.56,17880.66,12202.61,20825.57,412111.00,7316.69
3,1444.96,8222.03,5779.83,4060.00,81171.81,2552.00,2393.32,5819.63,8569.75,5848.40,9981.18,197514.48,3506.71



=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=


In [46]:
# Get the first sheet info
first_path, first_sheet, first_display = sheet_info[0]

# Read and process the data
df = pd.read_excel(first_path, sheet_name=first_sheet)

# Filter out unwanted IDG values and create pivot table
filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day

# Create comprehensive IDG pivot table function for reusability

def process_sheet_data_by_type(path, sheet_name, display_name, type_filter=None):
    """Process a single sheet and return IDG pivot by week, optionally filtered by TYPE"""
    df = pd.read_excel(path, sheet_name=sheet_name)
    filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
    
    # Apply TYPE filter if specified
    if type_filter:
        # Map CC to Jumbo.ae first
        filtered_df['TYPE'] = filtered_df['TYPE'].replace('CC', 'Jumbo.ae')
        filtered_df = filtered_df[filtered_df['TYPE'] == type_filter]
    
    filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day

    original_len = len(filtered_df)
    filtered_df = filtered_df[filtered_df['InvoiceDay'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_df)} rows with InvoiceDay > {max_invoice_day}")
    
    # Add week number calculation using simplified function
    filtered_df['WeekNumber'] = filtered_df['InvoiceDay'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    # Create pivot table with IDG on rows and weeks on columns
    idg_pivot = filtered_df.pivot_table(
        values='Amount Invoiced W.O. VAT',
        index='idg',
        columns='WeekNumber',
        aggfunc='sum',
        fill_value=0
    )
    
    return idg_pivot

def create_comprehensive_pivot_table(sheet_info, periods, type_filter=None, table_name="IDG"):
    """Create a comprehensive pivot table with weeks as super columns and periods as sub-columns, including target data"""
    
    print(f"\n🔄 Creating {table_name} Pivot Table with Target Data")
    print(f"🏷️ Type Filter: {type_filter or 'None (All Types)'}")
    
    # Process target data based on type filter
    # Map type_filter to appropriate channel for target data
    target_channel_filter = None
    if type_filter == "EA":
        target_channel_filter = "EA"
    elif type_filter == "Jumbo.ae":
        target_channel_filter = "Jumbo.ae"
    # For Overall analysis (type_filter=None), we use all channels
    
    print(f"🎯 Processing target data with channel filter: {target_channel_filter or 'ALL CHANNELS'}")
    target_by_week = process_target_data_by_week_and_channel(target_channel_filter)
    
    # Process all sheets with optional TYPE filter
    # Using display_name as the key in the dictionary instead of sheet_name
    sheet_data = {}
    for path, sheet_name, display_name in sheet_info:
        print(f"📄 Processing {display_name} data...")
        sheet_data[display_name] = process_sheet_data_by_type(path, sheet_name, display_name, type_filter)
    
    # Get all unique IDG values across all sheets and target data
    all_idgs = set()
    for data in sheet_data.values():
        all_idgs.update(data.index)
    
    # Add IDGs from target data if available
    if not target_by_week.empty:
        all_idgs.update(target_by_week.columns)
        print(f"🎯 Target IDGs found: {list(target_by_week.columns)}")
    else:
        print(f"⚠️ No target data available for filter: {target_channel_filter}")
    
    all_idgs = sorted(list(all_idgs))
    print(f"📋 Combined IDGs: {all_idgs}")
    
    # Calculate maximum week number across all data
    max_week = 1
    for data in sheet_data.values():
        if len(data.columns) > 0:
            max_week = max(max_week, max(data.columns))
    
    if not target_by_week.empty and len(target_by_week.index) > 0:
        max_week = max(max_week, max(target_by_week.index))
    
    print(f"📅 Maximum week number: {max_week}")
    
    # Create the comprehensive pivot table
    weeks = list(range(1, max_week + 1))
    
    # Create multi-level column index with target, comparison columns and totals
    # Order: Last Month, Last Year, Target, Current Month, vs Target %, vs Last Year %, vs Last Month %
    column_tuples = []
    for week in weeks:
        week_label = get_week_label(week)
        # Add period columns in specific order
        column_tuples.append((week_label, LAST_MONTH_DISPLAY))
        column_tuples.append((week_label, LAST_YEAR_DISPLAY))
        column_tuples.append((week_label, 'Target'))
        column_tuples.append((week_label, CURRENT_DISPLAY))
        # Add comparison columns
        column_tuples.append((week_label, 'v/s Target %'))
        column_tuples.append((week_label, 'v/s Last Year %'))
        column_tuples.append((week_label, 'v/s Last Month %'))
    # Add Total columns in same order
    column_tuples.append(('Total', LAST_MONTH_DISPLAY))
    column_tuples.append(('Total', LAST_YEAR_DISPLAY))
    column_tuples.append(('Total', 'Target'))
    column_tuples.append(('Total', CURRENT_DISPLAY))
    column_tuples.append(('Total', 'v/s Target %'))
    column_tuples.append(('Total', 'v/s Last Year %'))
    column_tuples.append(('Total', 'v/s Last Month %'))
    
    multi_columns = pd.MultiIndex.from_tuples(column_tuples, names=['Week', 'Period'])
    
    # Create the final dataframe
    final_df = pd.DataFrame(index=all_idgs, columns=multi_columns)
    
    # Fill the dataframe with data and calculate comparisons
    for idg in all_idgs:
        for week in weeks:
            week_label = get_week_label(week)
            
            # Fill period data
            for period in periods:
                if period in sheet_data and idg in sheet_data[period].index and week in sheet_data[period].columns:
                    final_df.loc[idg, (week_label, period)] = sheet_data[period].loc[idg, week]
                else:
                    final_df.loc[idg, (week_label, period)] = 0
            
            # Fill target data
            if not target_by_week.empty and week in target_by_week.index and idg in target_by_week.columns:
                final_df.loc[idg, (week_label, 'Target')] = target_by_week.loc[week, idg]
            else:
                final_df.loc[idg, (week_label, 'Target')] = 0
            
            # Calculate comparison percentages for each IDG and week
            current_val = final_df.loc[idg, (week_label, CURRENT_DISPLAY)]
            last_year_val = final_df.loc[idg, (week_label, LAST_YEAR_DISPLAY)]
            last_month_val = final_df.loc[idg, (week_label, LAST_MONTH_DISPLAY)]
            target_val = final_df.loc[idg, (week_label, 'Target')]
            
            # v/s Target %
            if target_val != 0:
                vs_target = (current_val / target_val * 100)  # Changed formula
                final_df.loc[idg, (week_label, 'v/s Target %')] = round(vs_target, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Target %')] = 0 if current_val == 0 else float('inf') # Or handle as per requirement for 0 target
            
            # v/s Last Year %
            if last_year_val != 0:
                vs_last_year = ((current_val - last_year_val) / last_year_val * 100)
                final_df.loc[idg, (week_label, 'v/s Last Year %')] = round(vs_last_year, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Last Year %')] = 0 if current_val == 0 else float('inf')
            
            # v/s Last Month %
            if last_month_val != 0:
                vs_last_month = ((current_val - last_month_val) / last_month_val * 100)
                final_df.loc[idg, (week_label, 'v/s Last Month %')] = round(vs_last_month, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Last Month %')] = 0 if current_val == 0 else float('inf')
    
    # Fill NaN values with 0
    final_df = final_df.fillna(0)
    
    # Calculate Total columns for each IDG
    for idg in all_idgs:
        # Calculate totals for each period across all weeks
        for period in periods:
            period_cols = [col for col in final_df.columns if col[1] == period and col[0] != 'Total']
            total_value = final_df.loc[idg, period_cols].sum()
            final_df.loc[idg, ('Total', period)] = total_value
        
        # Calculate total for target
        target_cols = [col for col in final_df.columns if col[1] == 'Target' and col[0] != 'Total']
        total_target = final_df.loc[idg, target_cols].sum()
        final_df.loc[idg, ('Total', 'Target')] = total_target
        
        # Calculate total comparison percentages
        total_current = final_df.loc[idg, ('Total', CURRENT_DISPLAY)]
        total_last_year = final_df.loc[idg, ('Total', LAST_YEAR_DISPLAY)]
        total_last_month = final_df.loc[idg, ('Total', LAST_MONTH_DISPLAY)]
        total_target = final_df.loc[idg, ('Total', 'Target')]
        
        # Total v/s Target %
        if total_target != 0:
            total_vs_target = (total_current / total_target * 100)  # Changed formula
            final_df.loc[idg, ('Total', 'v/s Target %')] = round(total_vs_target, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Target %')] = 0 if total_current == 0 else float('inf') # Or handle as per requirement for 0 target
        
        # Total v/s Last Year %
        if total_last_year != 0:
            total_vs_last_year = ((total_current - total_last_year) / total_last_year * 100)
            final_df.loc[idg, ('Total', 'v/s Last Year %')] = round(total_vs_last_year, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Last Year %')] = 0 if total_current == 0 else float('inf')
        
        # Total v/s Last Month %
        if total_last_month != 0:
            total_vs_last_month = ((total_current - total_last_month) / total_last_month * 100)
            final_df.loc[idg, ('Total', 'v/s Last Month %')] = round(total_vs_last_month, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Last Month %')] = 0 if total_current == 0 else float('inf')
    
    # Add Total row for all IDGs combined
    total_row_data = {}
    for col in final_df.columns:
        if 'v/s' in col[1]:  # For percentage columns, calculate weighted averages
            if col[1] in ['v/s Target %', 'v/s Last Year %', 'v/s Last Month %']:
                # Calculate overall percentage for the total row
                if col[0] == 'Total':  # Total column
                    total_current = final_df[('Total', CURRENT_DISPLAY)].sum()
                    total_base = 0
                    if col[1] == 'v/s Target %':
                        total_base = final_df[('Total', 'Target')].sum()
                    elif col[1] == 'v/s Last Year %':
                        total_base = final_df[('Total', LAST_YEAR_DISPLAY)].sum()
                    else:  # v/s Last Month %
                        total_base = final_df[('Total', LAST_MONTH_DISPLAY)].sum()
                    
                    if total_base != 0:
                        if col[1] == 'v/s Target %':
                            total_percentage = (total_current / total_base * 100) # Changed formula
                        else:
                            total_percentage = ((total_current - total_base) / total_base * 100)
                        total_row_data[col] = round(total_percentage, 2)
                    else:
                        total_row_data[col] = 0
                else:  # Week-wise percentage columns
                    week_label = col[0]
                    week_current = final_df[(week_label, CURRENT_DISPLAY)].sum()
                    week_base = 0
                    if col[1] == 'v/s Target %':
                        week_base = final_df[(week_label, 'Target')].sum()
                    elif col[1] == 'v/s Last Year %':
                        week_base = final_df[(week_label, LAST_YEAR_DISPLAY)].sum()
                    else:  # v/s Last Month %
                        week_base = final_df[(week_label, LAST_MONTH_DISPLAY)].sum()
                    
                    if week_base != 0:
                        if col[1] == 'v/s Target %':
                            week_percentage = (week_current / week_base * 100) # Changed formula
                        else:
                            week_percentage = ((week_current - week_base) / week_base * 100)
                        total_row_data[col] = round(week_percentage, 2)
                    else:
                        total_row_data[col] = 0
        else:
            # For amount columns, sum all IDGs
            total_row_data[col] = final_df[col].sum()
    
    # Create total row as a DataFrame and concatenate
    total_row_df = pd.DataFrame([total_row_data], index=['Total'])
    final_df = pd.concat([final_df, total_row_df])
    
    print(f"✅ {table_name} pivot table created successfully!")
    print(f"📊 Final shape: {final_df.shape}")
    
    return final_df, all_idgs, max_week

# Create three pivot tables: Overall, EA only, and Jumbo.ae only with Target Data
print("🔄 Creating Multiple Pivot Tables WITH TARGET DATA AND PROPER FILTERING...")
print("=" * 70)

# 1. Overall IDG Pivot Table (all data)
print("\n📊 1. OVERALL IDG ANALYSIS (All Types) - WITH TARGET DATA")
print("-" * 55)
global_idg_pivot, all_idgs_global, max_week_global = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter=None, table_name="Overall IDG"
)

print(f"\n📊 Data Shape: {global_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_global)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_global} (Starting day: {start_day}) + Total column")

# Summary statistics by period for overall
print("\n📊 SUMMARY BY PERIOD (Overall):")
for period in periods:
    period_total = global_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total = global_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total:,.2f}")

print("\n📋 Overall IDG Pivot Table (with Target):")
display(global_idg_pivot.round(2))

# 2. EA Only Pivot Table
print("\n" + "=" * 70)
print("📊 2. EA ONLY ANALYSIS - WITH TARGET DATA")
print("-" * 45)
ea_idg_pivot, all_idgs_ea, max_week_ea = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter="EA", table_name="EA IDG"
)

print(f"\n📊 Data Shape: {ea_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_ea)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_ea} (Starting day: {start_day}) + Total column")

# Summary statistics by period for EA
print("\n📊 SUMMARY BY PERIOD (EA Only):")
for period in periods:
    period_total = ea_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total_ea = ea_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total_ea:,.2f}")

print("\n📋 EA Only IDG Pivot Table (with Target):")
display(ea_idg_pivot.round(2))

# 3. Jumbo.ae Only Pivot Table
print("\n" + "=" * 70)
print("📊 3. JUMBO.AE ONLY ANALYSIS - WITH TARGET DATA")
print("-" * 50)
jumbo_idg_pivot, all_idgs_jumbo, max_week_jumbo = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter="Jumbo.ae", table_name="Jumbo.ae IDG"
)

print(f"\n📊 Data Shape: {jumbo_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_jumbo)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_jumbo} (Starting day: {start_day}) + Total column")

# Summary statistics by period for Jumbo.ae
print("\n📊 SUMMARY BY PERIOD (Jumbo.ae Only):")
for period in periods:
    period_total = jumbo_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total_jumbo = jumbo_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total_jumbo:,.2f}")

print("\n📋 Jumbo.ae Only IDG Pivot Table (with Target):")
display(jumbo_idg_pivot.round(2))

print("\n" + "=" * 70)
print("✅ ALL PIVOT TABLES WITH TARGET DATA CREATED SUCCESSFULLY!")
print("📊 Summary:")
print(f"   • Overall Analysis: {len(all_idgs_global)} IDGs, {max_week_global} weeks")
print(f"   • EA Analysis: {len(all_idgs_ea)} IDGs, {max_week_ea} weeks")
print(f"   • Jumbo.ae Analysis: {len(all_idgs_jumbo)} IDGs, {max_week_jumbo} weeks")
print(f"   • Each table includes: {len(periods)} periods + Target + vs Target % + comparisons + totals")
print(f"   • Column order: Last Month, Last Year, Target, Current Month, vs Target %, vs Last Year %, vs Last Month %")
print("=" * 70)

🔄 Creating Multiple Pivot Tables WITH TARGET DATA AND PROPER FILTERING...

📊 1. OVERALL IDG ANALYSIS (All Types) - WITH TARGET DATA
-------------------------------------------------------

🔄 Creating Overall IDG Pivot Table with Target Data
🏷️ Type Filter: None (All Types)
🎯 Processing target data with channel filter: ALL CHANNELS

🔄 Processing target data for channel: ALL CHANNELS
📊 Original target data shape: (806, 5)
📉 June 25: Filtered 494 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']
📄 Processing May 25 data...
📉 May 25: Filtered 3217 rows with InvoiceDay > 12
📄 Processing June 24 data...
📉 May 25: Filtered 3217 rows with InvoiceDay > 12
📄 Processing June 24 data...
📉 

C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:174: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = final_df.fillna(0)
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '15145.167' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[idg, ('Total', period)] = total_value
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4405.168' has dtype incompatible with int64, please explicitly cast to a compatible dtyp

Week 1                                                \
          May 25    June 24     Target    June 25 v/s Target %   
ACC      2002.86     398.10    2548.43       0.00         0.00   
AV      14663.81    5098.48   11651.67       0.00         0.00   
GAM       289.52     836.19    7218.74     347.62         4.82   
IMG      8368.57    2568.57    5460.00    1808.62        33.12   
IT      66954.33   38434.75   48381.92   42469.85        87.78   
LA       3237.67       0.00    5692.00       0.00         0.00   
MM       1614.28    1760.00    4242.75     931.43        21.95   
OAD     13075.24    3951.43    5306.09       0.00         0.00   
PG       6663.31    1284.76    4759.58       0.00         0.00   
PRO         0.00      14.29       0.00       0.00         0.00   
S&N       570.48    3903.81    6288.45       0.00         0.00   
SDA      4177.16    3998.10    7997.61       0.00         0.00   
TEL     90445.28  244661.91  117195.09   93771.88        80.01   
UNW       380.95       0.00       0.00       0.00         0.00   
WTC      1664.76    4516.19    3888.29    3289.75        84.61   
Total  214108.21  311426.57  230630.62  142619.15        61.84   

                                            Week 2                          \
      v/s Last Year % v/s Last Month %      May 25     June 24      Target   
ACC           -100.00          -100.00     9511.83     3124.22    17705.20   
AV            -100.00          -100.00    42612.70    84293.36    80813.22   
GAM            -58.43            20.07     6239.76    26465.97    50009.52   
IMG            -29.59           -78.39    35871.54    25552.33    37851.77   
IT              10.50           -36.57   347989.35   201419.75   331591.59   
LA               0.00          -100.00    23175.04    14527.62    39602.13   
MM             -47.08           -42.30    23026.76    10196.29    29477.49   
OAD           -100.00          -100.00    37197.33    15578.70    36626.43   
PG            -100.00          -100.00    11483.38     7261.95    32570.98   
PRO           -100.00             0.00        0.00      142.86        0.00   
S&N           -100.00          -100.00    21206.67     9838.62    43495.99   
SDA           -100.00          -100.00    25415.39    34598.36    55102.99   
TEL            -61.67             3.68   721658.42   667784.48   803135.85   
UNW              0.00          -100.00     2683.81        0.00        0.00   
WTC            -27.16            97.61    18538.14    15124.96    26903.79   
Total          -54.20           -33.39  1326610.11  1115909.47  1584886.95   

       ...       Week 3                                        Total  \
       ... v/s Target % v/s Last Year % v/s Last Month %      May 25   
ACC    ...        60.56          544.08            56.63    15145.17   
AV     ...       124.63        17395.90            56.89    91155.55   
GAM    ...       145.87          105.10           334.14    15367.90   
IMG    ...        45.36          -44.89           -55.29    64480.40   
IT     ...        72.87           38.42           -17.13   563466.65   
LA     ...        81.91            1.33            83.34    35834.61   
MM     ...        69.40           92.84           -11.97    36966.95   
OAD    ...        41.22            0.84            18.65    56895.47   
PG     ...        29.31           48.19           -37.60    25904.23   
PRO    ...         0.00         -100.00             0.00        0.00   
S&N    ...        97.13          277.74            78.50    34168.57   
SDA    ...        48.58           93.67           -46.30    55395.19   
TEL    ...        68.86            6.23           -29.84  1213550.71   
UNW    ...          inf             inf             1.52     3693.33   
WTC    ...       117.70          155.24           155.06    26709.57   
Total  ...        74.34           36.35           -13.32  2238734.29   

                                                                        \
          June 24      Target     June 25 v/s


📊 2. EA ONLY ANALYSIS - WITH TARGET DATA
---------------------------------------------

🔄 Creating EA IDG Pivot Table with Target Data
🏷️ Type Filter: EA
🎯 Processing target data with channel filter: EA

🔄 Processing target data for channel: EA
📊 Original target data shape: (806, 5)
🔍 After EA filter: (403, 5)
📉 June 25: Filtered 247 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']
📄 Processing May 25 data...
📉 May 25: Filtered 1556 rows with InvoiceDay > 12
📄 Processing June 24 data...
📉 May 25: Filtered 1556 rows with InvoiceDay > 12
📄 Processing June 24 data...
📉 June 24: Filtered 1408 rows with InvoiceDay > 12
📄 Processing June 25 data...
📉 June 24: Filtered 1408 rows wit

C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:174: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = final_df.fillna(0)
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3095.2379999999994' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[idg, ('Total', period)] = total_value
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '260.409' has dtype incompatible with int64, please explicitly cast to a compati

Week 1                                                               \
         May 25    June 24     Target   June 25 v/s Target % v/s Last Year %   
ACC       75.24       0.00     448.43      0.00         0.00            0.00   
AV     11236.19    1766.10    2551.67      0.00         0.00         -100.00   
GAM        0.00       0.00    1793.74      0.00         0.00            0.00   
IMG        0.00       0.00    1260.00   1808.62       143.54             inf   
IT     20761.92   27982.47   25191.25  21123.68        83.85          -24.51   
LA       858.62       0.00     792.00      0.00         0.00            0.00   
MM       522.86     618.10     742.75    856.19       115.27           38.52   
OAD     1903.81    3263.81    1806.09      0.00         0.00         -100.00   
PG      5199.07       0.00    2659.58      0.00         0.00            0.00   
S&N        0.00       0.00    1815.02      0.00         0.00            0.00   
SDA      559.07    1999.05    3097.61      0.00         0.00         -100.00   
TEL    41411.67  240379.05   61297.60  72184.61       117.76          -69.97   
WTC        0.00    2853.33    1088.29   3289.75       302.29           15.30   
Total  82528.44  278861.90  104544.03  99262.85        94.95          -64.40   

                           Week 2                        ...       Week 3  \
      v/s Last Month %     May 25    June 24     Target  ... v/s Target %   
ACC            -100.00    2355.24     185.17    3014.88  ...        58.99   
AV             -100.00    8235.55   16327.62   17155.15  ...        21.88   
GAM               0.00    4736.19    5425.71   12059.52  ...         0.00   
IMG                inf   11539.05    6970.97    8471.13  ...        39.85   
IT                1.74  177332.61  142419.09  169363.76  ...        55.81   
LA             -100.00    4457.20       0.00    5324.71  ...         0.00   
MM               63.75    4934.28    1962.00    4993.62  ...       147.88   
OAD            -100.00   13730.48    3081.90   12142.56  ...        31.88   
PG             -100.00    1618.10    3709.57   17880.66  ...        17.96   
S&N               0.00    3369.52    4607.19   12202.61  ...        39.07   
SDA            -100.00   13914.58   11684.36   20825.57  ...        65.67   
TEL              74.31  303813.71  511559.58  412111.00  ...        77.81   
WTC                inf   10281.95    3963.90    7316.69  ...       121.68   
Total            20.28  560318.46  711897.08  702861.86  ...        66.29   

                                            Total                          \
      v/s Last Year % v/s Last Month %     May 25     June 24      Target   
ACC           1032.94            28.23    3095.24      260.41     4908.27   
AV                inf            -0.53   21280.32    18093.72    27928.85   
GAM           -100.00          -100.00    5451.00     7280.00    19633.08   
IMG            -80.29           -59.32   15517.14    15181.45    13791.13   
IT              -2.44           -51.87  292216.56   216831.12   275726.82   
LA            -100.00             0.00    5315.82     5492.79     8668.71   
MM              87.59           -26.01   10240.49     4466.76     8129.68   
OAD            129.45           -49.36   19298.14     7154.29    19768.28   
PG                inf           -22.17    8794.70     3709.57    29109.99   
S&N               inf           -15.80    6082.86     4607.19    19866.03   
SDA               inf           -58.69   30340.52    13683.41    33904.35   
TEL            -16.62           -20.85  539403.83   936265.54   670923.07   
WTC             18.53           -16.46   15389.57    10417.24    11911.69   
Total          -11.63           -32.25  972426.20  1243443.47  1144269.97   

                                                                 
          June 25 v/s Target % v/s Last Year % v/s Last Month %  
ACC       1675.85        34.14          543.54           -45.86  
AV        4834.29        17.31          -73.28           -77.28 


📊 3. JUMBO.AE ONLY ANALYSIS - WITH TARGET DATA
--------------------------------------------------

🔄 Creating Jumbo.ae IDG Pivot Table with Target Data
🏷️ Type Filter: Jumbo.ae
🎯 Processing target data with channel filter: Jumbo.ae

🔄 Processing target data for channel: Jumbo.ae
📊 Original target data shape: (806, 5)
🔍 After Jumbo.ae filter: (403, 5)
📉 June 25: Filtered 247 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categories: ['ACC' 'AV' 'GAM' 'IMG' 'IT' 'LA' 'MM' 'OAD' 'PG' 'S&N' 'SDA' 'TEL' 'WTC']
📊 Final target pivot shape: (3, 13)
📋 Target IDGs/Categories: ['ACC', 'AV', 'GAM', 'IMG', 'IT', 'LA', 'MM', 'OAD', 'PG', 'S&N', 'SDA', 'TEL', 'WTC']
📄 Processing May 25 data...
📊 Original target data shape: (806, 5)
🔍 After Jumbo.ae filter: (403, 5)
📉 June 25: Filtered 247 rows with Day > 12
📅 Date range in target: 1 to 12
📋 Week range: 1 to 3
🏷️ Using 'Category' column for IDG grouping
📅 Available categ

C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:174: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = final_df.fillna(0)
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12049.929' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[idg, ('Total', period)] = total_value
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\4090792534.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4144.759' has dtype incompatible with int64, please explicitly cast to a compatible dtyp

Week 1                                                              \
          May 25   June 24     Target   June 25 v/s Target % v/s Last Year %   
ACC      1927.62    398.10    2100.00      0.00         0.00         -100.00   
AV       3427.62   3332.38    9100.00      0.00         0.00         -100.00   
GAM       289.52    836.19    5425.00    347.62         6.41          -58.43   
IMG      8368.57   2568.57    4200.00      0.00         0.00         -100.00   
IT      46192.40  10452.28   23190.67  21346.17        92.05          104.22   
LA       2379.05      0.00    4900.00      0.00         0.00            0.00   
MM       1091.43   1141.90    3500.00     75.24         2.15          -93.41   
OAD     11171.43    687.62    3500.00      0.00         0.00         -100.00   
PG       1464.24   1284.76    2100.00      0.00         0.00         -100.00   
PRO         0.00     14.29       0.00      0.00         0.00         -100.00   
S&N       570.48   3903.81    4473.43      0.00         0.00         -100.00   
SDA      3618.10   1999.05    4900.00      0.00         0.00         -100.00   
TEL     49033.61   4282.86   55897.49  21587.27        38.62          404.04   
UNW       380.95      0.00       0.00      0.00         0.00            0.00   
WTC      1664.76   1662.86    2800.00      0.00         0.00         -100.00   
Total  131579.77  32564.67  126086.59  43356.30        34.39           33.14   

                           Week 2                        ...       Week 3  \
      v/s Last Month %     May 25    June 24     Target  ... v/s Target %   
ACC            -100.00    7156.59    2939.05   14690.32  ...        60.84   
AV             -100.00   34377.14   67965.74   63658.06  ...       149.16   
GAM              20.07    1503.57   21040.26   37950.00  ...       186.95   
IMG            -100.00   24332.50   18581.36   29380.65  ...        46.77   
IT              -53.79  170656.74   59000.66  162227.82  ...        88.65   
LA             -100.00   18717.84   14527.62   34277.42  ...        93.18   
MM              -93.11   18092.47    8234.29   24483.87  ...        55.21   
OAD            -100.00   23466.85   12496.79   24483.87  ...        45.33   
PG             -100.00    9865.29    3552.38   14690.32  ...        41.56   
PRO               0.00       0.00     142.86       0.00  ...         0.00   
S&N            -100.00   17837.14    5231.43   31293.38  ...       117.19   
SDA            -100.00   11500.81   22914.00   34277.42  ...        39.39   
TEL             -55.97  417844.71  156224.90  391024.85  ...        60.50   
UNW            -100.00    2683.81       0.00       0.00  ...          inf   
WTC            -100.00    8256.19   11161.05   19587.10  ...       116.38   
Total           -67.05  766291.65  404012.39  882025.09  ...        80.02   

                                             Total                         \
      v/s Last Year % v/s Last Month %      May 25    June 24      Target   
ACC            498.53            62.99    12049.93    4144.76    24735.48   
AV           16803.74            60.13    69875.24   71601.94   107187.10   
GAM            127.67           372.34     9916.91   38730.83    63900.00   
IMG             -9.48           -54.30    48963.25   29360.41    49470.97   
IT              83.07            42.99   271250.09  111942.25   273158.31   
LA              49.49            83.34    30518.79   26082.95    57716.13   
MM              95.49            -3.06    26726.45   13116.19    41225.81   
OAD            -14.05           102.86    37597.33   20168.22    41225.81   
PG               1.08           -42.87    17109.52    8103.81    24735.48   
PRO           -100.00             0.00        0.00     285.72        0.00   
S&N            238.72           104.94    28085.72   14990.92    52691.62   
SDA              2.06           -26.51    25054.66   32067.33    57716.13   
TEL             58.35           -38.27   674146.87  241306.69   658405.48   
UNW               inf      


✅ ALL PIVOT TABLES WITH TARGET DATA CREATED SUCCESSFULLY!
📊 Summary:
   • Overall Analysis: 15 IDGs, 3 weeks
   • EA Analysis: 13 IDGs, 3 weeks
   • Jumbo.ae Analysis: 15 IDGs, 3 weeks
   • Each table includes: 3 periods + Target + vs Target % + comparisons + totals
   • Column order: Last Month, Last Year, Target, Current Month, vs Target %, vs Last Year %, vs Last Month %


In [47]:
# IDG Analysis Report - Multiple Pivot Tables

# Additional pivot table views for all three analyses

# Weekly Analysis Views

# 1. Show first few weeks detailed view for Overall analysis
print("📊 OVERALL ANALYSIS - First 3 Weeks IDG Data")
print("=" * 50)
first_weeks_cols = []
for week in range(1, min(4, max_week_global + 1)):  # First 3 weeks or max available
    week_label = get_week_label(week)
    for period in periods:  # Use dynamic periods instead of hardcoded values
        first_weeks_cols.append((week_label, period))

if first_weeks_cols:  # Only if we have data
    first_weeks_data = global_idg_pivot[first_weeks_cols]
    display(first_weeks_data.round(2))
else:
    print("No week data available")

print("\n" + "=" * 50)

# 2. Weekly totals comparison for Overall analysis
print("📈 OVERALL ANALYSIS - Weekly Totals Comparison")
print("=" * 45)
weekly_totals = pd.DataFrame(index=range(1, max_week_global + 1), columns=periods)  # Use dynamic periods

for week in range(1, max_week_global + 1):
    week_label = get_week_label(week)
    for period in periods:  # Use dynamic periods
        week_period_cols = [(week_label, period)]
        weekly_total = global_idg_pivot[week_period_cols].sum().sum()
        weekly_totals.loc[week, period] = weekly_total

# Convert to numeric and add growth percentages
weekly_totals = weekly_totals.astype(float)
weekly_totals['Current_vs_LastYear_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_YEAR_DISPLAY]) / weekly_totals[LAST_YEAR_DISPLAY] * 100).round(2)
weekly_totals['Current_vs_LastMonth_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_MONTH_DISPLAY]) / weekly_totals[LAST_MONTH_DISPLAY] * 100).round(2)

# Add week labels for better readability
weekly_totals.index = [get_week_label(week) for week in range(1, max_week_global + 1)]

display(weekly_totals.round(2))

print("\n" + "=" * 50)

# 3. IDG totals by period (summary pivot) for Overall analysis
print("📋 OVERALL ANALYSIS - IDG Totals by Period (Week-based)")
print("=" * 55)
idg_period_summary = pd.DataFrame(index=all_idgs_global, columns=periods)  # Use dynamic periods

for idg in all_idgs_global:
    for period in periods:  # Use dynamic periods
        period_cols = [col for col in global_idg_pivot.columns if col[1] == period]
        idg_period_summary.loc[idg, period] = global_idg_pivot.loc[idg, period_cols].sum()

# Convert to numeric and add totals
idg_period_summary = idg_period_summary.astype(float)
idg_period_summary['Total'] = idg_period_summary.sum(axis=1)
idg_period_summary.loc['Total'] = idg_period_summary.sum()

display(idg_period_summary.round(2))

# 4. Quick comparison between the three analyses
print("\n" + "=" * 60)
print("📊 COMPARISON SUMMARY - All Three Analyses")
print("=" * 60)

# Create summary comparison
comparison_summary = pd.DataFrame({
    'Analysis_Type': ['Overall', 'EA Only', 'Jumbo.ae Only'],
    'IDG_Count': [len(all_idgs_global), len(all_idgs_ea), len(all_idgs_jumbo)],
    'Max_Weeks': [max_week_global, max_week_ea, max_week_jumbo]
})

# Add period totals for each analysis
for period in periods:
    comparison_summary[f'{period}_Total'] = [
        global_idg_pivot.loc['Total', ('Total', period)],
        ea_idg_pivot.loc['Total', ('Total', period)],
        jumbo_idg_pivot.loc['Total', ('Total', period)]
    ]

print("Summary of All Three Pivot Table Analyses:")
display(comparison_summary.round(2))

print("\n📈 Key Insights:")
print(f"• Overall analysis covers {len(all_idgs_global)} IDG categories across {max_week_global} weeks")
print(f"• EA analysis covers {len(all_idgs_ea)} IDG categories across {max_week_ea} weeks")
print(f"• Jumbo.ae analysis covers {len(all_idgs_jumbo)} IDG categories across {max_week_jumbo} weeks")
print(f"• Each analysis includes {len(periods)} periods: {', '.join(periods)}")
print("=" * 60)

📊 OVERALL ANALYSIS - First 3 Weeks IDG Data


Week 1                            Week 2                          \
          May 25    June 24    June 25      May 25     June 24     June 25   
ACC      2002.86     398.10       0.00     9511.83     3124.22     7052.97   
AV      14663.81    5098.48       0.00    42612.70    84293.36    28331.45   
GAM       289.52     836.19     347.62     6239.76    26465.97    51039.00   
IMG      8368.57    2568.57    1808.62    35871.54    25552.33    27404.98   
IT      66954.33   38434.75   42469.85   347989.35   201419.75   362094.22   
LA       3237.67       0.00       0.00    23175.04    14527.62    19709.68   
MM       1614.28    1760.00     931.43    23026.76    10196.29    35584.72   
OAD     13075.24    3951.43       0.00    37197.33    15578.70    22137.17   
PG       6663.31    1284.76       0.00    11483.38     7261.95    10207.70   
PRO         0.00      14.29       0.00        0.00      142.86      175.24   
S&N       570.48    3903.81       0.00    21206.67     9838.62    51026.75   
SDA      4177.16    3998.10       0.00    25415.39    34598.36    33908.69   
TEL     90445.28  244661.91   93771.88   721658.42   667784.48   901104.11   
UNW       380.95       0.00       0.00     2683.81        0.00     1019.05   
WTC      1664.76    4516.19    3289.75    18538.14    15124.96    24818.86   
Total  214108.21  311426.57  142619.15  1326610.11  1115909.47  1575614.58   

          Week 3                        
          May 25    June 24    June 25  
ACC      3630.48     882.86    5686.26  
AV      33879.04     303.81   53154.30  
GAM      8838.62   18708.66   38372.14  
IMG     20240.28   16420.95    9050.28  
IT     148522.97   88918.87  123084.34  
LA       9421.90   17048.12   17274.44  
MM      12325.90    5626.66   10850.61  
OAD      6622.91    7792.38    7858.10  
PG       7757.54    3266.67    4840.97  
PRO         0.00     128.57       0.00  
S&N     12391.43    5855.68   22119.19  
SDA     25802.64    7154.28   13855.85  
TEL    401447.01  265125.84  281635.76  
UNW       628.57       0.00     638.10  
WTC      6506.67    6501.91   16595.64  
Total  698015.97  443735.27  605015.97


📈 OVERALL ANALYSIS - Weekly Totals Comparison


,May 25,June 24,June 25,Current_vs_LastYear_%,Current_vs_LastMonth_%
Week 1,428216.42,622853.13,285238.30,-54.20,-33.39
Week 2,2653220.23,2231818.94,3151229.16,41.20,18.77
Week 3,1396031.94,887470.54,1210031.94,36.35,-13.32



📋 OVERALL ANALYSIS - IDG Totals by Period (Week-based)


,May 25,June 24,June 25,Total
ACC,30290.33,8810.34,25478.45,64579.12
AV,182311.10,179391.30,162971.50,524673.91
GAM,30735.81,92021.65,179517.51,302274.97
IMG,128960.79,89083.71,76527.76,294572.26
IT,1126933.30,657546.75,1055296.82,2839776.87
LA,71669.22,63151.49,73968.23,208788.94
MM,73933.89,35165.90,94733.51,203833.31
OAD,113790.95,54645.02,59990.53,228426.50
PG,51808.46,23626.76,30097.35,105532.57
PRO,0.00,571.44,350.48,921.92



📊 COMPARISON SUMMARY - All Three Analyses
Summary of All Three Pivot Table Analyses:


,Analysis_Type,IDG_Count,Max_Weeks,May 25_Total,June 24_Total,June 25_Total
0,Overall,15,3,2238734.29,1871071.31,2323249.7
1,EA Only,13,3,972426.20,1243443.47,1228833.6
2,Jumbo.ae Only,15,3,1266308.10,627627.84,1094416.1



📈 Key Insights:
• Overall analysis covers 15 IDG categories across 3 weeks
• EA analysis covers 13 IDG categories across 3 weeks
• Jumbo.ae analysis covers 15 IDG categories across 3 weeks
• Each analysis includes 3 periods: May 25, June 24, June 25


In [48]:
# Weekly Comparison Analysis - Focused View for All Three Analyses
print("📊 COMPREHENSIVE GROWTH COMPARISON ANALYSIS")
print("=" * 55)

def analyze_growth_patterns(pivot_table, all_idgs, max_week, analysis_name):
    """Analyze growth patterns for a given pivot table"""
    print(f"\n🔍 {analysis_name.upper()} ANALYSIS - Growth Patterns")
    print("-" * 50)
    
    # Create a focused view showing only comparison percentages
    comparison_cols = []
    for week in range(1, max_week + 1):
        week_label = get_week_label(week)
        comparison_cols.extend([
            (week_label, 'v/s Last Year %'),
            (week_label, 'v/s Last Month %')
        ])
    
    if comparison_cols:
        comparison_data = pivot_table[comparison_cols]
        print(f"\n📈 {analysis_name} - Growth Percentages by Week:")
        display(comparison_data.round(2))
        
        # Calculate average growth rates
        print(f"\n📊 {analysis_name} - Average Growth Rates Across All Weeks:")
        print("-" * 45)
        
        # Get all v/s Last Year columns
        last_year_cols = [col for col in comparison_data.columns if 'v/s Last Year %' in col[1]]
        last_month_cols = [col for col in comparison_data.columns if 'v/s Last Month %' in col[1]]
        
        for idg in all_idgs:
            avg_vs_last_year = comparison_data.loc[idg, last_year_cols].replace([float('inf'), -float('inf')], 0).mean()
            avg_vs_last_month = comparison_data.loc[idg, last_month_cols].replace([float('inf'), -float('inf')], 0).mean()
            print(f"{idg:15s} | Avg vs Last Year: {avg_vs_last_year:6.1f}% | Avg vs Last Month: {avg_vs_last_month:6.1f}%")
        
        # Weekly summary of overall performance
        print(f"\n📋 {analysis_name} - Weekly Performance Summary (All IDGs Combined):")
        print("-" * 55)
        
        weekly_performance = pd.DataFrame(index=range(1, max_week + 1), 
                                        columns=['vs_Last_Year_%', 'vs_Last_Month_%'])
        
        for week in range(1, max_week + 1):
            week_label = get_week_label(week)
            
            # Calculate total amounts for the week using dynamic period names
            current_total = pivot_table[(week_label, CURRENT_DISPLAY)].sum()
            last_year_total = pivot_table[(week_label, LAST_YEAR_DISPLAY)].sum()
            last_month_total = pivot_table[(week_label, LAST_MONTH_DISPLAY)].sum()
            
            # Calculate overall percentage changes
            if last_year_total != 0:
                vs_last_year = ((current_total - last_year_total) / last_year_total * 100)
                weekly_performance.loc[week, 'vs_Last_Year_%'] = round(vs_last_year, 2)
            else:
                weekly_performance.loc[week, 'vs_Last_Year_%'] = 0
                
            if last_month_total != 0:
                vs_last_month = ((current_total - last_month_total) / last_month_total * 100)
                weekly_performance.loc[week, 'vs_Last_Month_%'] = round(vs_last_month, 2)
            else:
                weekly_performance.loc[week, 'vs_Last_Month_%'] = 0
        
        # Add week labels for better readability
        weekly_performance.index = [get_week_label(week) for week in range(1, max_week + 1)]
        
        display(weekly_performance)
        
        return weekly_performance
    else:
        print(f"No comparison data available for {analysis_name}")
        return None

# Analyze all three pivot tables
overall_performance = analyze_growth_patterns(global_idg_pivot, all_idgs_global, max_week_global, "Overall")
ea_performance = analyze_growth_patterns(ea_idg_pivot, all_idgs_ea, max_week_ea, "EA Only")
jumbo_performance = analyze_growth_patterns(jumbo_idg_pivot, all_idgs_jumbo, max_week_jumbo, "Jumbo.ae Only")

# Cross-analysis comparison
print("\n" + "=" * 70)
print("🔄 CROSS-ANALYSIS PERFORMANCE COMPARISON")
print("=" * 70)

if overall_performance is not None and ea_performance is not None and jumbo_performance is not None:
    # Compare average performance across all analyses
    cross_comparison = pd.DataFrame({
        'Analysis': ['Overall', 'EA Only', 'Jumbo.ae Only'],
        'Avg_Growth_vs_Last_Year_%': [
            overall_performance['vs_Last_Year_%'].mean(),
            ea_performance['vs_Last_Year_%'].mean(),
            jumbo_performance['vs_Last_Year_%'].mean()
        ],
        'Avg_Growth_vs_Last_Month_%': [
            overall_performance['vs_Last_Month_%'].mean(),
            ea_performance['vs_Last_Month_%'].mean(),
            jumbo_performance['vs_Last_Month_%'].mean()
        ]
    })
    
    print("📊 Average Growth Rates Comparison:")
    display(cross_comparison.round(2))
    
    # Best and worst performing analysis types
    best_vs_year = cross_comparison.loc[cross_comparison['Avg_Growth_vs_Last_Year_%'].idxmax(), 'Analysis']
    best_vs_month = cross_comparison.loc[cross_comparison['Avg_Growth_vs_Last_Month_%'].idxmax(), 'Analysis']
    
    print(f"\n✅ Performance Highlights:")
    print(f"• Best performing vs Last Year: {best_vs_year}")
    print(f"• Best performing vs Last Month: {best_vs_month}")
else:
    print("Could not complete cross-analysis comparison")

print("\n" + "=" * 70)
print("✅ COMPREHENSIVE GROWTH ANALYSIS COMPLETE")
print("📊 All three pivot tables analyzed for growth patterns")
print("📈 Weekly performance trends calculated and compared")
print("=" * 70)

📊 COMPREHENSIVE GROWTH COMPARISON ANALYSIS

🔍 OVERALL ANALYSIS - Growth Patterns
--------------------------------------------------

📈 Overall - Growth Percentages by Week:


Week 1                           Week 2                   \
      v/s Last Year % v/s Last Month % v/s Last Year % v/s Last Month %   
ACC           -100.00          -100.00          125.75           -25.85   
AV            -100.00          -100.00          -66.39           -33.51   
GAM            -58.43            20.07           92.85           717.96   
IMG            -29.59           -78.39            7.25           -23.60   
IT              10.50           -36.57           79.77             4.05   
LA               0.00          -100.00           35.67           -14.95   
MM             -47.08           -42.30          249.00            54.54   
OAD           -100.00          -100.00           42.10           -40.49   
PG            -100.00          -100.00           40.56           -11.11   
PRO           -100.00             0.00           22.66              inf   
S&N           -100.00          -100.00          418.64           140.62   
SDA           -100.00          -100.00           -1.99            33.42   
TEL            -61.67             3.68           34.94            24.87   
UNW              0.00          -100.00             inf           -62.03   
WTC            -27.16            97.61           64.09            33.88   
Total          -54.20           -33.39           41.20            18.77   

               Week 3                   
      v/s Last Year % v/s Last Month %  
ACC            544.08            56.63  
AV           17395.90            56.89  
GAM            105.10           334.14  
IMG            -44.89           -55.29  
IT              38.42           -17.13  
LA               1.33            83.34  
MM              92.84           -11.97  
OAD              0.84            18.65  
PG              48.19           -37.60  
PRO           -100.00             0.00  
S&N            277.74            78.50  
SDA             93.67           -46.30  
TEL              6.23           -29.84  
UNW               inf             1.52  
WTC            155.24           155.06  
Total           36.35           -13.32


📊 Overall - Average Growth Rates Across All Weeks:
---------------------------------------------
ACC             | Avg vs Last Year:  189.9% | Avg vs Last Month:  -23.1%
AV              | Avg vs Last Year: 5743.2% | Avg vs Last Month:  -25.5%
GAM             | Avg vs Last Year:   46.5% | Avg vs Last Month:  357.4%
IMG             | Avg vs Last Year:  -22.4% | Avg vs Last Month:  -52.4%
IT              | Avg vs Last Year:   42.9% | Avg vs Last Month:  -16.6%
LA              | Avg vs Last Year:   12.3% | Avg vs Last Month:  -10.5%
MM              | Avg vs Last Year:   98.3% | Avg vs Last Month:    0.1%
OAD             | Avg vs Last Year:  -19.0% | Avg vs Last Month:  -40.6%
PG              | Avg vs Last Year:   -3.8% | Avg vs Last Month:  -49.6%
PRO             | Avg vs Last Year:  -59.1% | Avg vs Last Month:    0.0%
S&N             | Avg vs Last Year:  198.8% | Avg vs Last Month:   39.7%
SDA             | Avg vs Last Year:   -2.8% | Avg vs Last Month:  -37.6%
TEL             | Avg vs L

,vs_Last_Year_%,vs_Last_Month_%
Week 1,-54.2,-33.39
Week 2,41.2,18.77
Week 3,36.35,-13.32



🔍 EA ONLY ANALYSIS - Growth Patterns
--------------------------------------------------

📈 EA Only - Growth Percentages by Week:


...


📊 Jumbo.ae Only - Average Growth Rates Across All Weeks:
---------------------------------------------
ACC             | Avg vs Last Year:  170.2% | Avg vs Last Month:  -16.7%
AV              | Avg vs Last Year: 5547.0% | Avg vs Last Month:  -22.1%
GAM             | Avg vs Last Year:   55.4% | Avg vs Last Month: 1015.9%
IMG             | Avg vs Last Year:  -41.4% | Avg vs Last Month:  -63.1%
IT              | Avg vs Last Year:  112.4% | Avg vs Last Month:   -8.1%
LA              | Avg vs Last Year:   23.1% | Avg vs Last Month:   -7.9%
MM              | Avg vs Last Year:   64.2% | Avg vs Last Month:  -21.3%
OAD             | Avg vs Last Year:  -36.3% | Avg vs Last Month:  -13.7%
PG              | Avg vs Last Year:   13.1% | Avg vs Last Month:  -52.4%
PRO             | Avg vs Last Year:  -59.1% | Avg vs Last Month:    0.0%
S&N             | Avg vs Last Year:  234.8% | Avg vs Last Month:   33.4%
SDA             | Avg vs Last Year:  -40.1% | Avg vs Last Month:  -24.0%
TEL             | Av

,vs_Last_Year_%,vs_Last_Month_%
Week 1,33.14,-67.05
Week 2,65.67,-12.65
Week 3,99.8,3.6



🔄 CROSS-ANALYSIS PERFORMANCE COMPARISON
📊 Average Growth Rates Comparison:


,Analysis,Avg_Growth_vs_Last_Year_%,Avg_Growth_vs_Last_Month_%
0,Overall,7.78,-9.31
1,EA Only,-16.24,16.59
2,Jumbo.ae Only,66.20,-25.37



✅ Performance Highlights:
• Best performing vs Last Year: Jumbo.ae Only
• Best performing vs Last Month: EA Only

✅ COMPREHENSIVE GROWTH ANALYSIS COMPLETE
📊 All three pivot tables analyzed for growth patterns
📈 Weekly performance trends calculated and compared


In [49]:
# Export All IDG Pivot Tables to Excel - Combined in Single Sheet
print("📤 EXPORTING ALL THREE PIVOT TABLES TO SINGLE EXCEL SHEET...")
print("=" * 65)

# Create Excel writer object
output_file = 'IDG_Weekly_Analysis_Combined.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Create combined sheet with all three analyses
    print("📊 Creating Combined IDG Analysis Sheet...")
    
    # Create a workbook and worksheet - ensure we have a proper worksheet
    workbook = writer.book
    # Remove default sheet if it exists
    if workbook.worksheets:
        workbook.remove(workbook.active)
    
    # Create our main worksheet
    worksheet = workbook.create_sheet('Combined_IDG_Analysis', 0)
    
    # Define styles for different sections
    main_title_font = Font(bold=True, size=14, color='FFFFFF')
    main_title_fill = PatternFill(start_color='1F4E79', end_color='1F4E79', fill_type='solid')
    
    section_header_font = Font(bold=True, size=12, color='FFFFFF')
    section_header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    
    column_header_font = Font(bold=True, size=10, color='FFFFFF')
    column_header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    
    idg_name_font = Font(bold=True, size=9)
    idg_name_fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
    
    number_format = '#,##0.00'
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )
    
    # MODIFIED Function to write a single pivot table to a dedicated worksheet
    def write_pivot_table_to_sheet(pivot_df, analysis_title, worksheet):
        # Section title for the sheet
        worksheet.cell(row=1, column=1, value=analysis_title)
        title_cell = worksheet.cell(row=1, column=1)
        title_cell.font = section_header_font # Use section_header_font for individual sheet titles
        title_cell.fill = section_header_fill
        title_cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Merge cells for the title across all columns of the pivot table
        num_cols = len(pivot_df.columns)
        worksheet.merge_cells(start_row=1, start_column=1, 
                            end_row=1, end_column=num_cols + 1) # +1 for the IDG column
        
        # Apply border to title
        for col_idx_merge in range(1, num_cols + 2):
            worksheet.cell(row=1, column=col_idx_merge).border = thin_border
        
        current_row = 2 # Start headers from row 2
        
        # Write column headers (multi-level)
        # First level headers (Week labels)
        col_idx = 2  # Start from column 2 (column 1 is for IDG names)
        for week_label in pivot_df.columns.get_level_values(0).unique():
            week_cols = [col for col in pivot_df.columns if col[0] == week_label]
            num_sub_cols = len(week_cols)
            
            worksheet.cell(row=current_row, column=col_idx, value=week_label)
            week_cell = worksheet.cell(row=current_row, column=col_idx)
            week_cell.font = column_header_font
            week_cell.fill = column_header_fill
            week_cell.alignment = Alignment(horizontal='center', vertical='center')
            week_cell.border = thin_border
            
            if num_sub_cols > 1:
                worksheet.merge_cells(start_row=current_row, start_column=col_idx,
                                    end_row=current_row, end_column=col_idx + num_sub_cols - 1)
                for i in range(num_sub_cols):
                    cell = worksheet.cell(row=current_row, column=col_idx + i)
                    cell.font = column_header_font
                    cell.fill = column_header_fill
                    cell.border = thin_border
            col_idx += num_sub_cols
        
        current_row += 1
        
        # Second level headers (Period names)
        worksheet.cell(row=current_row, column=1, value="IDG")
        idg_header_cell = worksheet.cell(row=current_row, column=1)
        idg_header_cell.font = column_header_font
        idg_header_cell.fill = column_header_fill
        idg_header_cell.alignment = Alignment(horizontal='center', vertical='center')
        idg_header_cell.border = thin_border
        
        col_idx = 2
        for col_multi_idx in pivot_df.columns:
            worksheet.cell(row=current_row, column=col_idx, value=col_multi_idx[1])
            sub_header_cell = worksheet.cell(row=current_row, column=col_idx)
            sub_header_cell.font = column_header_font
            sub_header_cell.fill = column_header_fill
            sub_header_cell.alignment = Alignment(horizontal='center', vertical='center')
            sub_header_cell.border = thin_border
            col_idx += 1
        
        current_row += 1
        
        # Write data rows
        for idg_name in pivot_df.index:
            worksheet.cell(row=current_row, column=1, value=idg_name)
            name_cell = worksheet.cell(row=current_row, column=1)
            if idg_name == 'Total':
                name_cell.font = Font(bold=True, size=10)
                name_cell.fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
            else:
                name_cell.font = idg_name_font
                name_cell.fill = idg_name_fill
            name_cell.alignment = Alignment(horizontal='left', vertical='center')
            name_cell.border = thin_border
            
            col_idx = 2
            for col_multi_idx in pivot_df.columns:
                value = pivot_df.loc[idg_name, col_multi_idx]
                worksheet.cell(row=current_row, column=col_idx, value=value)
                data_cell = worksheet.cell(row=current_row, column=col_idx)
                
                if idg_name == 'Total':
                    data_cell.font = Font(bold=True, size=9)
                    data_cell.fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
                else:
                    data_cell.font = Font(size=9)
                
                if 'v/s' in col_multi_idx[1] and '%' in col_multi_idx[1]:
                    data_cell.number_format = '0.00"%"'
                    if isinstance(value, (int, float)) and value != float('inf') and value != float('-inf') and value != 0 : # Check for valid numeric value
                        if col_multi_idx[1] == 'v/s Target %':
                            if value >= 100:
                                data_cell.font = Font(color='008000', size=9, bold=idg_name=='Total')
                            else:
                                data_cell.font = Font(color='FF0000', size=9, bold=idg_name=='Total')
                        else:
                            if value > 0:
                                data_cell.font = Font(color='008000', size=9, bold=idg_name=='Total')
                            elif value < 0:
                                data_cell.font = Font(color='FF0000', size=9, bold=idg_name=='Total')
                else:
                    data_cell.number_format = number_format
                
                data_cell.alignment = Alignment(horizontal='right', vertical='center')
                data_cell.border = thin_border
                col_idx += 1
            current_row += 1
            
        # Auto-adjust column widths for this worksheet
        for col_letter_idx in range(1, worksheet.max_column + 1):
            max_length = 0
            column_letter = get_column_letter(col_letter_idx)
            for row_idx in range(1, worksheet.max_row + 1):
                try:
                    cell_value = str(worksheet.cell(row=row_idx, column=col_letter_idx).value)
                    if len(cell_value) > max_length:
                        max_length = len(cell_value)
                except:
                    pass
            adjusted_width = min(max_length + 2, 30) # Max width of 30, increased padding
            worksheet.column_dimensions[column_letter].width = adjusted_width

        # Freeze panes for this worksheet (IDG name and headers)
        # Freeze row 3 (headers) and column 1 (IDG names)
        worksheet.freeze_panes = 'B1' # Top 3 rows and 1st column frozen

    # Main Excel writing block
    # Remove default sheet if it exists
    if workbook.worksheets:
        workbook.remove(workbook.active)
    
    # Data for sheets
    analyses_to_export = [
        {"name": "Overall_IDG_Analysis", "title": f"OVERALL IDG ANALYSIS ({max_invoice_day} days)", "data": global_idg_pivot},
        {"name": "EA_IDG_Analysis", "title": f"EA ONLY IDG ANALYSIS ({max_invoice_day} days)", "data": ea_idg_pivot},
        {"name": "JumboAE_IDG_Analysis", "title": f"JUMBO.AE ONLY IDG ANALYSIS ({max_invoice_day} days)", "data": jumbo_idg_pivot},
    ]

    sheet_idx_counter = 0
    for analysis in analyses_to_export:
        print(f"  📊 Writing {analysis['title']} to sheet: {analysis['name']}...")
        ws = workbook.create_sheet(analysis['name'], sheet_idx_counter)
        write_pivot_table_to_sheet(analysis['data'].round(2), analysis['title'], ws)
        sheet_idx_counter += 1
    
    # Create additional supporting sheets (Summary Dashboard, Weekly Totals)
    # These will be created after the individual analysis sheets.
    
    # 2. Summary Dashboard Sheet
    print("📊 Creating Summary Dashboard...")
    summary_data = {
        'Analysis_Type': ['Overall', 'EA Only', 'Jumbo.ae Only'],
        'IDG_Categories': [len(all_idgs_global), len(all_idgs_ea), len(all_idgs_jumbo)],
        'Weeks_Analyzed': [max_week_global, max_week_ea, max_week_jumbo]
    }
    
    # Add period totals for each analysis
    for period in periods:
        summary_data[f'{period}_Total'] = [
            global_idg_pivot.loc['Total', ('Total', period)],
            ea_idg_pivot.loc['Total', ('Total', period)],
            jumbo_idg_pivot.loc['Total', ('Total', period)]
        ]
    
    # Add growth percentages
    summary_data[f'Growth_vs_{LAST_YEAR_DISPLAY}_%'] = [
        global_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')],
        ea_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')],
        jumbo_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')]
    ]
    
    summary_data[f'Growth_vs_{LAST_MONTH_DISPLAY}_%'] = [
        global_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')],
        ea_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')],
        jumbo_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')]
    ]
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.round(2).to_excel(writer, sheet_name='Summary_Dashboard', index=False)
    
    # 3. Weekly Totals Summary (Overall)
    print("📊 Creating Weekly Totals Summary...")
    weekly_totals = pd.DataFrame(index=range(1, max_week_global + 1), columns=periods)
    
    for week in range(1, max_week_global + 1):
        week_label = get_week_label(week)
        for period in periods:
            week_period_cols = [(week_label, period)]
            weekly_total = global_idg_pivot[week_period_cols].sum().sum()
            weekly_totals.loc[week, period] = weekly_total
    
    # Add growth percentages using dynamic period names
    weekly_totals = weekly_totals.astype(float)
    weekly_totals['Growth_vs_LastYear_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_YEAR_DISPLAY]) / weekly_totals[LAST_YEAR_DISPLAY] * 100).round(2)
    weekly_totals['Growth_vs_LastMonth_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_MONTH_DISPLAY]) / weekly_totals[LAST_MONTH_DISPLAY] * 100).round(2)
    
    # Add week labels
    weekly_totals.index = [get_week_label(week) for week in range(1, max_week_global + 1)]
    
    # Export weekly totals
    weekly_totals.round(2).to_excel(writer, sheet_name='Weekly_Totals', index=True)

print(f"\n✅ EXCEL EXPORT COMPLETED SUCCESSFULLY!")
print(f"📎 File: {output_file}")
print("\n📊 SHEETS CREATED:")
print("=" * 50)
print("📄 1. Overall_IDG_Analysis - Overall IDG Analysis table")
print("📄 2. EA_IDG_Analysis - EA Only IDG Analysis table")
print("📄 3. JumboAE_IDG_Analysis - Jumbo.ae Only IDG Analysis table")
print(f"📄 4. Summary_Dashboard - Overview of all analyses") # Adjusted numbering
print(f"📄 5. Weekly_Totals - Weekly summary with growth")   # Adjusted numbering

print(f"\n🎆 FEATURES APPLIED TO EACH ANALYSIS SHEET:")
print(f"• Dedicated sheet for each analysis: Overall, EA, Jumbo.ae")
print(f"• Professional multi-level header formatting")
print(f"• Color-coded percentages (Green: positive, Red: negative)")
print(f"• Highlighted Total columns and rows")
print(f"• Auto-adjusted column widths for readability")
print(f"• Frozen panes for easy navigation")
print(f"• Consistent number formatting across all sheets")
print(f"• Comprehensive analysis covering all three data types")

print(f"\n📊 ANALYSIS COVERAGE:")
print(f"• Overall: {len(all_idgs_global)} IDGs across {max_week_global} weeks")
print(f"• EA Only: {len(all_idgs_ea)} IDGs across {max_week_ea} weeks")
print(f"• Jumbo.ae: {len(all_idgs_jumbo)} IDGs across {max_week_jumbo} weeks")
print(f"• Periods: {', '.join(periods)}")
print(f"• Week calculation based on: {start_day} as first day")

print(f"\n💾 File saved as: {output_file}")
print("=" * 60)

📤 EXPORTING ALL THREE PIVOT TABLES TO SINGLE EXCEL SHEET...
📊 Creating Combined IDG Analysis Sheet...
  📊 Writing OVERALL IDG ANALYSIS (12 days) to sheet: Overall_IDG_Analysis...
📊 Creating Combined IDG Analysis Sheet...
  📊 Writing OVERALL IDG ANALYSIS (12 days) to sheet: Overall_IDG_Analysis...
  📊 Writing EA ONLY IDG ANALYSIS (12 days) to sheet: EA_IDG_Analysis...
  📊 Writing EA ONLY IDG ANALYSIS (12 days) to sheet: EA_IDG_Analysis...
  📊 Writing JUMBO.AE ONLY IDG ANALYSIS (12 days) to sheet: JumboAE_IDG_Analysis...  📊 Writing JUMBO.AE ONLY IDG ANALYSIS (12 days) to sheet: JumboAE_IDG_Analysis...

📊 Creating Summary Dashboard...
📊 Creating Weekly Totals Summary...
📊 Creating Summary Dashboard...
📊 Creating Weekly Totals Summary...

✅ EXCEL EXPORT COMPLETED SUCCESSFULLY!
📎 File: IDG_Weekly_Analysis_Combined.xlsx

📊 SHEETS CREATED:
📄 1. Overall_IDG_Analysis - Overall IDG Analysis table
📄 2. EA_IDG_Analysis - EA Only IDG Analysis table
📄 3. JumboAE_IDG_Analysis - Jumbo.ae Only IDG Anal

In [50]:
# Dynamic constants for sessions
LAST_MONTH_SESSION_PATH = sessions_info[0][0]
LAST_MONTH_SESSION_SHEET = sessions_info[0][1]
LAST_MONTH_SESSION_DISPLAY = sessions_info[0][2]

LAST_YEAR_SESSION_PATH = sessions_info[1][0]
LAST_YEAR_SESSION_SHEET = sessions_info[1][1]
LAST_YEAR_SESSION_DISPLAY = sessions_info[1][2]

CURRENT_SESSION_PATH = sessions_info[2][0]
CURRENT_SESSION_SHEET = sessions_info[2][1]
CURRENT_SESSION_DISPLAY = sessions_info[2][2]

# Metrics to include in the pivot table
METRICS_TO_AGGREGATE = ['Sessions', 'Purchases', 'Purchase revenue']

def create_master_sessions_pivot(sessions_info, cg_filter=None):
    """
    Create master sessions pivot table with optional CG column filtering
    
    Parameters:
    sessions_info: List of tuples containing (path, sheet, display_name)
    cg_filter: String indicating filter type:
               - None: No filter (default)
               - "EA_only": Only include "EA" or "Endless Aisle" 
               - "non_EA": Exclude "EA" or "Endless Aisle"
    
    Returns:
    pd.DataFrame: Master pivot table with calculated metrics
    """
    
    print("📊 SESSIONS DATA ANALYSIS - WEEKLY MASTER PIVOT")
    if cg_filter == "EA_only":
        print("🔍 Filter: EA/Endless Aisle ONLY")
    elif cg_filter == "non_EA":
        print("🔍 Filter: NON-EA/Endless Aisle")
    else:
        print("🔍 Filter: NO FILTER")
    print("=" * 60)

    processed_sessions_data = {}
    all_channels = set()
    max_week_overall = 0

    # Read and process sessions data from all three periods
    for i, (path, sheet, display_name) in enumerate(sessions_info):
        print(f"\n📄 Processing {display_name} Sessions Data:")
        print("-" * 40)

        try:
            # Read the sessions data
            sessions_df = pd.read_excel(path, sheet_name=sheet)
            print(f"  Raw Shape: {sessions_df.shape}")
            print(f"  Raw Columns: {list(sessions_df.columns)}")
            
            # Filter out "Gift Card" from Category column if it exists
            if 'Category' in sessions_df.columns:
                sessions_df = sessions_df[sessions_df['Category'] != 'Gift Card ']
                print(f"  Shape after filtering 'Gift Card': {sessions_df.shape}")
            
            # Apply CG filter if specified
            if cg_filter and 'CG' in sessions_df.columns:
                if cg_filter == "EA_only":
                    sessions_df = sessions_df[sessions_df['CG'].isin(['EA', 'Endless Aisle'])]
                    print(f"  Shape after EA filter: {sessions_df.shape}")
                elif cg_filter == "non_EA":
                    sessions_df = sessions_df[~sessions_df['CG'].isin(['EA', 'Endless Aisle'])]
                    print(f"  Shape after non-EA filter: {sessions_df.shape}")
            elif cg_filter and 'CG' not in sessions_df.columns:
                print(f"  Warning: CG column not found, filter '{cg_filter}' cannot be applied")
            
            # Convert the string to datetime format first
            sessions_df['Date'] = pd.to_datetime(sessions_df['Date'], format='%Y%m%d', errors='coerce')

            # Then extract the day
            sessions_df['Day'] = sessions_df['Date'].dt.day
            


            sessions_df = sessions_df.dropna(subset=['Day']) 
            if sessions_df.empty:
                print(f"  No valid 'Day' data after conversion for {display_name}.")
                processed_sessions_data[display_name] = pd.DataFrame()
                continue

            sessions_df['Day'] = sessions_df['Day'].astype(int)

            original_len = len(sessions_df)
            sessions_df = sessions_df[sessions_df['Day'] <= max_invoice_day]
            print(f"📉 {display_name}: Filtered {original_len - len(sessions_df)} rows with Day > {max_invoice_day}")

            # Add WeekNumber column (ensure get_week_number and first_day_position are defined in a previous cell)
            sessions_df['WeekNumber'] = sessions_df['Day'].apply(
                lambda day: get_week_number(day, first_day_position) 
            )
            
            # Check which of the desired metrics are available in the current DataFrame
            available_metrics = [m for m in METRICS_TO_AGGREGATE if m in sessions_df.columns]
            if not available_metrics:
                print(f"  No metrics ({', '.join(METRICS_TO_AGGREGATE)}) found in {display_name}. Skipping.")
                processed_sessions_data[display_name] = pd.DataFrame()
                continue
            
            print(f"  Available metrics for {display_name}: {available_metrics}")

            # Select relevant columns for aggregation
            cols_for_aggregation = ['Channel', 'WeekNumber'] + available_metrics
            temp_df = sessions_df[cols_for_aggregation].copy()

            # Group by Channel and WeekNumber, and sum the available metrics
            aggregated_data_for_period = temp_df.groupby(['Channel', 'WeekNumber'])[available_metrics].sum().fillna(0)
            
            processed_sessions_data[display_name] = aggregated_data_for_period
            if not aggregated_data_for_period.empty:
                all_channels.update(aggregated_data_for_period.index.get_level_values('Channel').unique())
                # Ensure WeekNumber exists in index before calling max()
                if 'WeekNumber' in aggregated_data_for_period.index.names:
                    current_max_week_in_period = aggregated_data_for_period.index.get_level_values('WeekNumber').max()
                    if pd.notna(current_max_week_in_period) and current_max_week_in_period > max_week_overall:
                        max_week_overall = int(current_max_week_in_period)
                else: # Handle case where WeekNumber might not be in index (e.g. if groupby results in empty df for some reason)
                     current_max_week_in_period = 0
            else:
                current_max_week_in_period = 0
            
            print(f"  Processed {display_name} successfully. Max week: {current_max_week_in_period if current_max_week_in_period > 0 else 'N/A'}")

        except Exception as e:
            print(f"❌ Error processing {display_name}: {e}")
            import traceback
            traceback.print_exc()
            processed_sessions_data[display_name] = pd.DataFrame()

    print("\n" + "=" * 60)
    print("📊 CREATING FINAL COMPREHENSIVE SESSIONS MASTER PIVOT TABLE")

    if not processed_sessions_data or not all_channels:
        print("⚠️ No session data successfully processed or no channels found. Cannot create master pivot table.")
        return pd.DataFrame()
    
    sorted_channels = sorted(list(all_channels))
    session_periods_display_names = [LAST_MONTH_SESSION_DISPLAY, LAST_YEAR_SESSION_DISPLAY, CURRENT_SESSION_DISPLAY]
    
    column_tuples = []
    if max_week_overall > 0:
        weeks_for_pivot = list(range(1, int(max_week_overall) + 1))
        for week_num in weeks_for_pivot:
            week_label = get_week_label(week_num) # Ensure get_week_label is available
            for metric_name in METRICS_TO_AGGREGATE: # Use the full list of desired metrics
                for period_display_name in session_periods_display_names:
                    column_tuples.append((week_label, metric_name, period_display_name))
    
        if not column_tuples:
             print("⚠️ No weeks or metrics to create columns for. Pivot table will be empty or incomplete.")
             master_sessions_pivot_df = pd.DataFrame(index=sorted_channels)
        else:
            multi_columns = pd.MultiIndex.from_tuples(column_tuples, names=['Week', 'Metric', 'Period'])
            master_sessions_pivot_df = pd.DataFrame(index=sorted_channels, columns=multi_columns)
            master_sessions_pivot_df = master_sessions_pivot_df.fillna(0) # Initialize with 0

            for channel_val in sorted_channels:
                for week_num in weeks_for_pivot:
                    week_label = get_week_label(week_num)
                    for period_display_name in session_periods_display_names:
                        if period_display_name in processed_sessions_data:
                            period_aggregated_data = processed_sessions_data[period_display_name]
                            if not period_aggregated_data.empty and (channel_val, week_num) in period_aggregated_data.index:
                                data_series_for_channel_week = period_aggregated_data.loc[(channel_val, week_num)]
                                for metric_name in METRICS_TO_AGGREGATE:
                                    if metric_name in data_series_for_channel_week.index: # Check if metric was available for this period
                                        value = data_series_for_channel_week[metric_name]
                                        master_sessions_pivot_df.loc[channel_val, (week_label, metric_name, period_display_name)] = value
    else:
        print("⚠️ No weeks found in session data across all periods. Cannot create master pivot table.")
        return pd.DataFrame()

    # Calculate CVR and AOV if the DataFrame is not empty and has the required structure
    if not master_sessions_pivot_df.empty and isinstance(master_sessions_pivot_df.columns, pd.MultiIndex) and master_sessions_pivot_df.columns.nlevels == 3:

        # Collect unique (week_label, period_name) combinations that have base metrics
        processed_combinations = set()
        for col_tuple in master_sessions_pivot_df.columns:
            week_label, metric, period_name = col_tuple
            if metric in METRICS_TO_AGGREGATE:
                processed_combinations.add((week_label, period_name))

        for week_label, period_name in processed_combinations:
            sessions_col_tuple = (week_label, 'Sessions', period_name)
            purchases_col_tuple = (week_label, 'Purchases', period_name)
            revenue_col_tuple = (week_label, 'Purchase revenue', period_name)

            # Check if all base metric columns exist for this combination
            if not (sessions_col_tuple in master_sessions_pivot_df.columns and \
                    purchases_col_tuple in master_sessions_pivot_df.columns and \
                    revenue_col_tuple in master_sessions_pivot_df.columns):
                continue

            sessions_s = master_sessions_pivot_df[sessions_col_tuple]
            purchases_s = master_sessions_pivot_df[purchases_col_tuple]
            purchase_revenue_s = master_sessions_pivot_df[revenue_col_tuple]

            # Calculate CVR (Purchases / Sessions)
            cvr = purchases_s / sessions_s
            master_sessions_pivot_df[(week_label, 'CVR', period_name)] = cvr.fillna(0).replace([float('inf'), -float('inf')], 0)

            # Calculate AOV (Purchase Revenue / Purchases)
            aov = purchase_revenue_s / purchases_s
            master_sessions_pivot_df[(week_label, 'AOV', period_name)] = aov.fillna(0).replace([float('inf'), -float('inf')], 0)

        # Re-sort columns to ensure CVR and AOV are placed correctly and maintain overall structure
        all_metrics_ordered = METRICS_TO_AGGREGATE + ['CVR', 'AOV']
        
        # Get unique week labels and sort them
        unique_week_labels_sorted = sorted(list(master_sessions_pivot_df.columns.get_level_values('Week').unique()))

        # Get ordered periods
        period_order_from_sessions_info = [s_info[2] for s_info in sessions_info] 
        actual_periods_in_df = list(master_sessions_pivot_df.columns.get_level_values('Period').unique())
        ordered_periods = [p for p in period_order_from_sessions_info if p in actual_periods_in_df]
        for p_df in actual_periods_in_df:
            if p_df not in ordered_periods:
                ordered_periods.append(p_df)

        new_column_tuples = []
        for week_l in unique_week_labels_sorted:
            for metric_n in all_metrics_ordered:
                for period_dn in ordered_periods: 
                    # Check if this week/metric/period combination had any original data or newly added CVR/AOV
                    if (week_l, metric_n, period_dn) in master_sessions_pivot_df.columns:
                        new_column_tuples.append((week_l, metric_n, period_dn))
        
        if new_column_tuples:
            master_sessions_pivot_df = master_sessions_pivot_df.reindex(columns=pd.MultiIndex.from_tuples(new_column_tuples))
    
    # Add Grand Total row
    if not master_sessions_pivot_df.empty:
        print("📊 Adding Grand Total row...")
        
        # Create grand total row
        grand_total_row = pd.DataFrame(index=['Grand Total'], columns=master_sessions_pivot_df.columns)
        
        # Calculate totals for each column
        for col in master_sessions_pivot_df.columns:
            week_label, metric, period_name = col
            
            if metric in ['Sessions', 'Purchases', 'Purchase revenue']:
                # Sum these metrics
                grand_total_row.loc['Grand Total', col] = master_sessions_pivot_df[col].sum()
            elif metric == 'CVR':
                # Calculate overall CVR = Total Purchases / Total Sessions
                total_sessions_col = (week_label, 'Sessions', period_name)
                total_purchases_col = (week_label, 'Purchases', period_name)
                
                if total_sessions_col in master_sessions_pivot_df.columns and total_purchases_col in master_sessions_pivot_df.columns:
                    total_sessions = master_sessions_pivot_df[total_sessions_col].sum()
                    total_purchases = master_sessions_pivot_df[total_purchases_col].sum()
                    
                    if total_sessions > 0:
                        grand_total_row.loc['Grand Total', col] = total_purchases / total_sessions
                    else:
                        grand_total_row.loc['Grand Total', col] = 0
                else:
                    grand_total_row.loc['Grand Total', col] = 0
            elif metric == 'AOV':
                # Calculate overall AOV = Total Revenue / Total Purchases
                total_revenue_col = (week_label, 'Purchase revenue', period_name)
                total_purchases_col = (week_label, 'Purchases', period_name)
                
                if total_revenue_col in master_sessions_pivot_df.columns and total_purchases_col in master_sessions_pivot_df.columns:
                    total_revenue = master_sessions_pivot_df[total_revenue_col].sum()
                    total_purchases = master_sessions_pivot_df[total_purchases_col].sum()
                    
                    if total_purchases > 0:
                        grand_total_row.loc['Grand Total', col] = total_revenue / total_purchases
                    else:
                        grand_total_row.loc['Grand Total', col] = 0
                else:
                    grand_total_row.loc['Grand Total', col] = 0
        
        # Append grand total row to the main dataframe
        master_sessions_pivot_df = pd.concat([master_sessions_pivot_df, grand_total_row.fillna(0)])
        print("✅ Grand Total row added successfully")
    
    return master_sessions_pivot_df.fillna(0)

# Generate all three pivot tables
print("🚀 GENERATING ALL THREE PIVOT TABLES")
print("=" * 80)

# 1. No filter pivot
print("\n1️⃣ CREATING NO FILTER PIVOT TABLE")
master_sessions_pivot_no_filter = create_master_sessions_pivot(sessions_info, cg_filter=None)

# 2. EA only pivot
print("\n2️⃣ CREATING EA ONLY PIVOT TABLE")
master_sessions_pivot_ea_only = create_master_sessions_pivot(sessions_info, cg_filter="EA_only")

# 3. Non-EA pivot
print("\n3️⃣ CREATING NON-EA PIVOT TABLE")
master_sessions_pivot_non_ea = create_master_sessions_pivot(sessions_info, cg_filter="non_EA")

# Display all three pivot tables
print("\n" + "=" * 80)
print("📊 DISPLAYING ALL PIVOT TABLES")
print("=" * 80)

print("\n🔍 NO FILTER PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_no_filter)

print("\n🔍 EA ONLY PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_ea_only)

print("\n🔍 NON-EA PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_non_ea)

🚀 GENERATING ALL THREE PIVOT TABLES

1️⃣ CREATING NO FILTER PIVOT TABLE
📊 SESSIONS DATA ANALYSIS - WEEKLY MASTER PIVOT
🔍 Filter: NO FILTER

📄 Processing May 25 Sessions Data:
----------------------------------------
  Raw Shape: (9555, 16)
  Raw Columns: ['Day', 'Combined', 'Date', 'Transaction ID', 'Session campaign', 'Session Channel Group - 1', 'Channel', 'Session source / medium', 'Sessions', 'Purchases', 'Purchase revenue', 'CG', 'Category', 'Unnamed: 13', 20250501, datetime.datetime(2025, 5, 1, 0, 0)]
  Shape after filtering 'Gift Card': (8774, 16)
📉 May 25: Filtered 5490 rows with Day > 12
  Available metrics for May 25: ['Sessions', 'Purchases', 'Purchase revenue']
  Processed May 25 successfully. Max week: 3

📄 Processing June 24 Sessions Data:
----------------------------------------
  Raw Shape: (9555, 16)
  Raw Columns: ['Day', 'Combined', 'Date', 'Transaction ID', 'Session campaign', 'Session Channel Group - 1', 'Channel', 'Session source / medium', 'Sessions', 'Purchases'

Traceback (most recent call last):
  File "C:\Users\91843\AppData\Roaming\Python\Python312\site-packages\pandas\core\indexes\base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Date'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\91843\AppData\Local\Temp\ipykernel_30936\2021629137.py", line 73, in create_master_sessions_pivot
    sessions_df['Date'] = pd.to_datetime(sessions_df['Date'], format='%Y%m%d', errors='coerce')
                       

  Raw Shape: (9555, 16)
  Raw Columns: ['Day', 'Combined', 'Date', 'Transaction ID', 'Session campaign', 'Session Channel Group - 1', 'Channel', 'Session source / medium', 'Sessions', 'Purchases', 'Purchase revenue', 'CG', 'Category', 'Unnamed: 13', 20250601, datetime.datetime(2025, 6, 1, 0, 0)]
  Shape after filtering 'Gift Card': (9555, 16)
📉 June 25: Filtered 0 rows with Day > 12
  Available metrics for June 25: ['Sessions', 'Purchases', 'Purchase revenue']
  Processed June 25 successfully. Max week: 3

📊 CREATING FINAL COMPREHENSIVE SESSIONS MASTER PIVOT TABLE
📊 Adding Grand Total row...
✅ Grand Total row added successfully

2️⃣ CREATING EA ONLY PIVOT TABLE
📊 SESSIONS DATA ANALYSIS - WEEKLY MASTER PIVOT
🔍 Filter: EA/Endless Aisle ONLY

📄 Processing May 25 Sessions Data:
----------------------------------------


C:\Users\91843\AppData\Local\Temp\ipykernel_30936\2021629137.py:159: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  master_sessions_pivot_df = master_sessions_pivot_df.fillna(0) # Initialize with 0
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\2021629137.py:172: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '19642.999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  master_sessions_pivot_df.loc[channel_val, (week_label, metric_name, period_display_name)] = value
C:\Users\91843\AppData\Local\Temp\ipykernel_30936\2021629137.py:172: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of p

  Raw Shape: (9555, 16)
  Raw Columns: ['Day', 'Combined', 'Date', 'Transaction ID', 'Session campaign', 'Session Channel Group - 1', 'Channel', 'Session source / medium', 'Sessions', 'Purchases', 'Purchase revenue', 'CG', 'Category', 'Unnamed: 13', 20250501, datetime.datetime(2025, 5, 1, 0, 0)]
  Shape after filtering 'Gift Card': (8774, 16)
  Shape after EA filter: (1577, 16)
📉 May 25: Filtered 990 rows with Day > 12
  Available metrics for May 25: ['Sessions', 'Purchases', 'Purchase revenue']
  Processed May 25 successfully. Max week: 3

📄 Processing June 24 Sessions Data:
----------------------------------------
  Raw Shape: (9323, 11)
  Raw Columns: ['# ----------------------------------------', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10']
❌ Error processing June 24: 'Date'

📄 Processing June 25 Sessions Data:
----------------------------------------
  Raw Shape: (9323, 11)
  Raw Column

...

Week 1                                            \
            Sessions                 Purchases                   
              May 25 June 24 June 25    May 25 June 24 June 25   
Affiliates      1012       0     717         7       0       1   
Display          689       0      28         0       0       0   
Email            170       0    1513         0       0       5   
Organic        13974       0   17993        61       0      36   
Paid Other        15       0     143         0       0       0   
Paid Perf       6917       0    8706        29       0      18   
Paid Social        3       0       3         0       0       0   
Referral           0       0       0         0       0       0   
Grand Total    22780       0   29103        97       0      60   

                                                               ...    Week 3  \
            Purchase revenue                              CVR  ... Purchases   
                      May 25 June 24        June 25    May 25  ...   June 25   
Affiliates      19642.999999       0    1199.000000  0.006917  ...        12   
Display             0.000000       0       0.000000  0.000000  ...         0   
Email               0.000000       0    4938.000004  0.000000  ...         2   
Organic        195768.489993       0   87858.000014  0.004365  ...       100   
Paid Other          0.000000       0       0.000000  0.000000  ...         0   
Paid Perf       52385.850016       0   34743.000005  0.004193  ...        87   
Paid Social         0.000000       0       0.000000  0.000000  ...         0   
Referral            0.000000       0       0.000000  0.000000  ...         1   
Grand Total    267797.340008       0  128738.000023  0.004258  ...       202   

                                                                       \
            Purchase revenue                              CVR           
                      May 25 June 24        June 25    May 25 June 24   
Affiliates      16811.999999       0   32608.000000  0.003214     0.0   
Display             0.000000       0       0.000000  0.000000     0.0   
Email            6923.000001       0    3818.000000  0.001319     0.0   
Organic        507395.680020       0  239845.090004  0.004543     0.0   
Paid Other          0.000000       0       0.000000  0.000000     0.0   
Paid Perf      262589.970003       0  152157.649999  0.005193     0.0   
Paid Social      1099.000001       0       0.000000  0.003257     0.0   
Referral            0.000000       0     999.000000  0.000000     0.0   
Grand Total    794819.650024       0  429427.740003  0.004174     0.0   

                                                         
                               AOV                       
              June 25       May 25 June 24      June 25  
Affiliates   0.004313  2401.714286     0.0  2717.333333  
Display      0.000000     0.000000     0.0     0.000000  
Email        0.001335  1153.833333     0.0  1909.000000  
Organic      0.001349  2062.584065     0.0  2398.450900  
Paid Other   0.000000     0.000000     0.0     0.000000  
Paid Perf    0.003219  1889.136475     0.0  1748.938506  
Paid Social  0.000000  1099.000001     0.0     0.000000  
Referral     0.023256     0.000000     0.0   999.000000  
Grand Total  0.001912  1992.029198     0.0  2125.879901  

[9 rows x 45 columns]


🔍 EA ONLY PIVOT TABLE:
----------------------------------------


Week 1                                            \
            Sessions                 Purchases                   
              May 25 June 24 June 25    May 25 June 24 June 25   
Affiliates         0       0       0         0       0       0   
Email              0       0       0         0       0       0   
Organic          159       0     208        28       0      23   
Paid Perf          4       0       7         4       0       7   
Grand Total      163       0     215        32       0      30   

                                                              ...    Week 3  \
            Purchase revenue                             CVR  ... Purchases   
                      May 25 June 24       June 25    May 25  ...   June 25   
Affiliates          0.000000       0      0.000000  0.000000  ...         1   
Email               0.000000       0      0.000000  0.000000  ...         1   
Organic         49259.639994       0  67376.000012  0.176101  ...        50   
Paid Perf        8696.000002       0  14693.000001  1.000000  ...        25   
Grand Total     57955.639996       0  82069.000013  0.196319  ...        77   

                                                                       \
            Purchase revenue                              CVR           
                      May 25 June 24        June 25    May 25 June 24   
Affiliates       2599.000000       0    4498.999999  1.000000     0.0   
Email            2398.000002       0    3248.999999  1.000000     0.0   
Organic        279792.680007       0  108977.490004  0.210604     0.0   
Paid Perf       70729.919993       0   60710.849998  1.000000     0.0   
Grand Total    355519.600002       0  177436.340000  0.258645     0.0   

                                                         
                               AOV                       
              June 25       May 25 June 24      June 25  
Affiliates   1.000000  2599.000000     0.0  4498.999999  
Email        1.000000  1199.000001     0.0  3248.999999  
Organic      0.068776  1956.592168     0.0  2179.549800  
Paid Perf    1.000000  1725.120000     0.0  2428.434000  
Grand Total  0.102122  1901.174332     0.0  2304.368052  

[5 rows x 45 columns]


🔍 NON-EA PIVOT TABLE:
----------------------------------------


Week 1                                            \
            Sessions                 Purchases                   
              May 25 June 24 June 25    May 25 June 24 June 25   
Affiliates      1012       0     717         7       0       1   
Display          689       0      28         0       0       0   
Email            170       0    1513         0       0       5   
Organic        13815       0   17785        33       0      13   
Paid Other        15       0     143         0       0       0   
Paid Perf       6913       0    8699        25       0      11   
Paid Social        3       0       3         0       0       0   
Referral           0       0       0         0       0       0   
Grand Total    22617       0   28888        65       0      30   

                                                              ...    Week 3  \
            Purchase revenue                             CVR  ... Purchases   
                      May 25 June 24       June 25    May 25  ...   June 25   
Affiliates      19642.999999       0   1199.000000  0.006917  ...        11   
Display             0.000000       0      0.000000  0.000000  ...         0   
Email               0.000000       0   4938.000004  0.000000  ...         1   
Organic        146508.849999       0  20482.000002  0.002389  ...        50   
Paid Other          0.000000       0      0.000000  0.000000  ...         0   
Paid Perf       43689.850014       0  20050.000004  0.003616  ...        62   
Paid Social         0.000000       0      0.000000  0.000000  ...         0   
Referral            0.000000       0      0.000000  0.000000  ...         1   
Grand Total    209841.700012       0  46669.000010  0.002874  ...       125   

                                                                       \
            Purchase revenue                              CVR           
                      May 25 June 24        June 25    May 25 June 24   
Affiliates      14212.999999       0   28109.000001  0.002756     0.0   
Display             0.000000       0       0.000000  0.000000     0.0   
Email            4524.999999       0     569.000001  0.000880     0.0   
Organic        227603.000013       0  130867.600000  0.001926     0.0   
Paid Other          0.000000       0       0.000000  0.000000     0.0   
Paid Perf      191860.050010       0   91446.800001  0.003667     0.0   
Paid Social      1099.000001       0       0.000000  0.003257     0.0   
Referral            0.000000       0     999.000000  0.000000     0.0   
Grand Total    439300.050022       0  251991.400003  0.002235     0.0   

                                                         
                               AOV                       
              June 25       May 25 June 24      June 25  
Affiliates   0.003955  2368.833333     0.0  2555.363636  
Display      0.000000     0.000000     0.0     0.000000  
Email        0.000668  1131.250000     0.0   569.000001  
Organic      0.000681  2209.737864     0.0  2617.352000  
Paid Other   0.000000     0.000000     0.0     0.000000  
Paid Perf    0.002296  1957.755612     0.0  1474.948387  
Paid Social  0.000000  1099.000001     0.0     0.000000  
Referral     0.023256     0.000000     0.0   999.000000  
Grand Total  0.001192  2072.170047     0.0  2015.931200  

[9 rows x 45 columns]

In [51]:
# Export sessions pivot tables to Excel with professional formatting
print("📤 EXPORTING BEAUTIFULLY FORMATTED SESSIONS PIVOT TABLES TO EXCEL...")
print("=" * 80)

output_file = 'IDG_Weekly_Analysis_Combined.xlsx'

try:
    # Load existing workbook
    from openpyxl import load_workbook
    from openpyxl.styles import Font, PatternFill, Border, Side, Alignment, numbers
    from openpyxl.utils import get_column_letter
    wb = load_workbook(output_file)
    
    # Define styles for formatting consistency with previous tables
    section_header_font = Font(bold=True, size=14, color='FFFFFF')
    section_header_fill = PatternFill(start_color='1F4E79', end_color='1F4E79', fill_type='solid')
    
    column_header_font = Font(bold=True, size=10, color='FFFFFF')
    column_header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    
    subheader_font = Font(bold=True, size=10, color='FFFFFF')
    subheader_fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
    
    row_header_font = Font(bold=True, size=9)
    row_header_fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
    
    total_row_font = Font(bold=True, size=10)
    total_row_fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
    
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )
    
    def write_sessions_pivot_to_sheet(pivot_df, sheet_name):
        print(f"Writing beautifully formatted sessions data to sheet: {sheet_name}")
        
        if sheet_name in wb.sheetnames:
            # Get the existing sheet
            ws = wb[sheet_name]
            
            # Find the first empty row (assuming data starts from row 1)
            last_row = ws.max_row
            start_row = last_row + 3  # Leave two blank rows for spacing
            
            # Determine column spans
            total_columns = len(pivot_df.columns) + 1  # +1 for the row headers column
            
            # Write section header with professional formatting
            ws.cell(row=start_row, column=1, value="SESSIONS ANALYSIS")
            header_cell = ws.cell(row=start_row, column=1)
            header_cell.font = section_header_font
            header_cell.fill = section_header_fill
            header_cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Merge cells for the header
            ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=total_columns)
            
            # Apply borders to all merged header cells
            for col_idx in range(1, total_columns + 1):
                cell = ws.cell(row=start_row, column=col_idx)
                cell.border = thin_border
            
            # Track current row after header
            current_row = start_row + 1
            
            # Create true multi-level headers (3 levels: Week, Metric, Period)
            
            # First, organize the columns into a hierarchy: Week -> Metric -> Period
            header_hierarchy = {}
            for col in pivot_df.columns:
                week, metric, period = col
                if week not in header_hierarchy:
                    header_hierarchy[week] = {}
                if metric not in header_hierarchy[week]:
                    header_hierarchy[week][metric] = []
                header_hierarchy[week][metric].append(period)
            
            # Row 1: Week headers (top level)
            col_idx = 2  # Start from column 2 (column 1 is for row labels)
            week_start_columns = {}  # To track where each week starts
            
            for week in header_hierarchy:
                week_start_columns[week] = col_idx
                
                # Calculate total columns for this week
                week_total_cols = 0
                for metric in header_hierarchy[week]:
                    week_total_cols += len(header_hierarchy[week][metric])
                
                # Write week header and merge cells
                ws.cell(row=current_row, column=col_idx, value=week)
                week_cell = ws.cell(row=current_row, column=col_idx)
                week_cell.font = column_header_font
                week_cell.fill = column_header_fill
                week_cell.alignment = Alignment(horizontal='center', vertical='center')
                week_cell.border = thin_border
                
                # Merge cells if needed
                if week_total_cols > 1:
                    ws.merge_cells(start_row=current_row, start_column=col_idx, 
                                  end_row=current_row, end_column=col_idx + week_total_cols - 1)
                    
                    # Apply styles to all merged cells
                    for i in range(week_total_cols):
                        merged_cell = ws.cell(row=current_row, column=col_idx + i)
                        merged_cell.border = thin_border
                        merged_cell.font = column_header_font
                        merged_cell.fill = column_header_fill
                
                # Move to next position
                col_idx += week_total_cols
            
            # Add "Channel" header for first column
            ws.cell(row=current_row, column=1, value="Channel")
            channel_header = ws.cell(row=current_row, column=1)
            channel_header.font = column_header_font
            channel_header.fill = column_header_fill
            channel_header.alignment = Alignment(horizontal='center', vertical='center')
            channel_header.border = thin_border
            ws.merge_cells(start_row=current_row, start_column=1, 
                          end_row=current_row + 2, end_column=1)  # Merge across all 3 header rows
            
            # Row 2: Metric headers (middle level)
            current_row += 1
            
            for week in header_hierarchy:
                metric_start_col = week_start_columns[week]
                
                for metric in header_hierarchy[week]:
                    # Calculate how many columns this metric spans
                    metric_periods = header_hierarchy[week][metric]
                    metric_cols = len(metric_periods)
                    
                    # Write metric header
                    ws.cell(row=current_row, column=metric_start_col, value=metric)
                    metric_cell = ws.cell(row=current_row, column=metric_start_col)
                    metric_cell.font = column_header_font
                    metric_cell.fill = subheader_fill  # Slightly different shade than the week
                    metric_cell.alignment = Alignment(horizontal='center', vertical='center')
                    metric_cell.border = thin_border
                    
                    # Merge cells if needed
                    if metric_cols > 1:
                        ws.merge_cells(start_row=current_row, start_column=metric_start_col, 
                                      end_row=current_row, end_column=metric_start_col + metric_cols - 1)
                        
                        # Apply styles to all merged cells
                        for i in range(metric_cols):
                            merged_cell = ws.cell(row=current_row, column=metric_start_col + i)
                            merged_cell.border = thin_border
                            merged_cell.font = column_header_font
                            merged_cell.fill = subheader_fill
                    
                    # Move to next position
                    metric_start_col += metric_cols
            
            # Row 3: Period headers (bottom level)
            current_row += 1
            col_idx = 2  # Reset column index
            
            for col in pivot_df.columns:
                period = col[2]  # Third level
                
                # Write period header
                ws.cell(row=current_row, column=col_idx, value=period)
                period_cell = ws.cell(row=current_row, column=col_idx)
                period_cell.font = subheader_font
                period_cell.fill = PatternFill(start_color='B4C6E7', end_color='B4C6E7', fill_type='solid')  # Lighter blue
                period_cell.alignment = Alignment(horizontal='center', vertical='center')
                period_cell.border = thin_border
                
                col_idx += 1
            
            # Move to next row for data
            current_row += 1
            
            # Write data rows
            for idx in pivot_df.index:
                # Write row header (Channel name)
                ws.cell(row=current_row, column=1, value=str(idx))
                row_header_cell = ws.cell(row=current_row, column=1)
                
                # Special formatting for Grand Total
                if idx == "Grand Total":
                    row_header_cell.font = total_row_font
                    row_header_cell.fill = total_row_fill
                else:
                    row_header_cell.font = row_header_font
                    row_header_cell.fill = row_header_fill
                    
                row_header_cell.alignment = Alignment(horizontal='left', vertical='center')
                row_header_cell.border = thin_border
                
                # Write data values
                col_idx = 2
                for col in pivot_df.columns:
                    value = pivot_df.loc[idx, col]
                    ws.cell(row=current_row, column=col_idx, value=value)
                    data_cell = ws.cell(row=current_row, column=col_idx)
                    
                    # Apply cell formatting based on content
                    if idx == "Grand Total":
                        data_cell.font = Font(bold=True, size=9)
                        data_cell.fill = total_row_fill
                    else:
                        data_cell.font = Font(size=9)
                    
                    # Format numbers based on metric type
                    metric = col[1]  # Second level (metric)
                    if 'CVR' in metric:
                        data_cell.number_format = '0.00%'
                        # No conditional coloring per user request
                    elif 'AOV' in metric:
                        data_cell.number_format = '#,##0.00'
                    elif 'revenue' in metric.lower():
                        data_cell.number_format = '#,##0.00'
                    else:
                        data_cell.number_format = '#,##0'
                    
                    # Apply borders and alignment
                    data_cell.alignment = Alignment(horizontal='right', vertical='center')
                    data_cell.border = thin_border
                    
                    col_idx += 1
                
                current_row += 1
            
            # Auto-adjust column widths for better readability
            for col_idx in range(1, total_columns + 1):
                col_letter = get_column_letter(col_idx)
                max_length = 0
                for row_idx in range(start_row, current_row):
                    cell = ws.cell(row=row_idx, column=col_idx)
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            if cell_length > max_length:
                                max_length = cell_length
                        except:
                            pass
                
                # Set width with padding, limit max width
                adjusted_width = min(max_length + 3, 25)
                ws.column_dimensions[col_letter].width = adjusted_width
            
            print(f"✅ Beautifully formatted sessions data written to {sheet_name} successfully!")
        else:
            print(f"❌ Sheet {sheet_name} not found in workbook")

    # Write each pivot table to its respective sheet
    write_sessions_pivot_to_sheet(master_sessions_pivot_no_filter, "Overall_IDG_Analysis")
    write_sessions_pivot_to_sheet(master_sessions_pivot_ea_only, "EA_IDG_Analysis")
    write_sessions_pivot_to_sheet(master_sessions_pivot_non_ea, "JumboAE_IDG_Analysis")

    # Save the workbook
    wb.save(output_file)
    print(f"\n✅ Successfully appended beautifully formatted sessions data to {output_file}")
    print("📊 Sessions analysis has been added to:")
    print("  • Overall_IDG_Analysis sheet")
    print("  • EA_IDG_Analysis sheet")
    print("  • JumboAE_IDG_Analysis sheet")
    
    print("\n🎨 Formatting Features Applied:")
    print("  • Professional color-coded headers and sections")
    print("  • Multi-level column headers with proper alignment")
    print("  • Highlighted Grand Total row")
    print("  • Color-coded CVR percentages (green/red)")
    print("  • Consistent borders and cell styling")
    print("  • Auto-adjusted column widths")
    print("  • Proper number formatting for different metrics")
    print("  • Clean visual separation between sections")

except Exception as e:
    print(f"❌ Error while exporting to Excel: {str(e)}")
    import traceback
    traceback.print_exc()  # Print full traceback for better debugging

print("\n" + "=" * 80)

📤 EXPORTING BEAUTIFULLY FORMATTED SESSIONS PIVOT TABLES TO EXCEL...
Writing beautifully formatted sessions data to sheet: Overall_IDG_Analysis
✅ Beautifully formatted sessions data written to Overall_IDG_Analysis successfully!
Writing beautifully formatted sessions data to sheet: EA_IDG_Analysis
Writing beautifully formatted sessions data to sheet: Overall_IDG_Analysis
✅ Beautifully formatted sessions data written to Overall_IDG_Analysis successfully!
Writing beautifully formatted sessions data to sheet: EA_IDG_Analysis
✅ Beautifully formatted sessions data written to EA_IDG_Analysis successfully!
Writing beautifully formatted sessions data to sheet: JumboAE_IDG_Analysis
✅ Beautifully formatted sessions data written to EA_IDG_Analysis successfully!
Writing beautifully formatted sessions data to sheet: JumboAE_IDG_Analysis
✅ Beautifully formatted sessions data written to JumboAE_IDG_Analysis successfully!

✅ Successfully appended beautifully formatted sessions data to IDG_Weekly_Analysi

In [52]:
# Optional: Open the comprehensive Excel file automatically (Windows)
import os
import subprocess

# Update to use the correct comprehensive file name
output_file = 'IDG_Weekly_Analysis_Combined.xlsx'

try:
    # Check if the file exists
    if os.path.exists(output_file):
        print(f"🚀 Opening {output_file} in Excel...")
        # Use the default program to open the Excel file
        os.startfile(output_file)
        print("✅ Excel file opened successfully!")
        print(f"📊 Navigate through the sheets to explore all analyses:")
        print(f"   • Combined_IDG_Analysis for comprehensive view")
        print(f"   • Summary Dashboard for quick overview")
        print(f"   • Weekly Totals for growth tracking")
    else:
        print(f"❌ File {output_file} not found!")
except Exception as e:
    print(f"⚠️ Could not open Excel file automatically: {e}")
    print(f"Please manually open: {output_file}")

print("\n" + "=" * 70)
print("🎉 COMPLETE IDG WEEKLY ANALYSIS EXPORT FINISHED!")
print("=" * 70)
print("📊 ANALYSIS SUMMARY:")
print(f"• 📈 Overall Analysis: {len(all_idgs_global)} IDGs across {max_week_global} weeks")
print(f"• 🏢 EA Only Analysis: {len(all_idgs_ea)} IDGs across {max_week_ea} weeks")
print(f"• 🛍️ Jumbo.ae Analysis: {len(all_idgs_jumbo)} IDGs across {max_week_jumbo} weeks")
print(f"• 📅 Periods Analyzed: {', '.join(periods)}")
print(f"• 🗺️ Week Calculation: Based on {start_day} as first day of month")
print(f"• 📎 Export File: {output_file}")
print("\n🏆 All pivot tables created with comprehensive analysis!")
print("📊 Ready for business insights and decision making!")
print("=" * 70)

🚀 Opening IDG_Weekly_Analysis_Combined.xlsx in Excel...
✅ Excel file opened successfully!
📊 Navigate through the sheets to explore all analyses:
   • Combined_IDG_Analysis for comprehensive view
   • Summary Dashboard for quick overview
   • Weekly Totals for growth tracking

🎉 COMPLETE IDG WEEKLY ANALYSIS EXPORT FINISHED!
📊 ANALYSIS SUMMARY:
• 📈 Overall Analysis: 15 IDGs across 3 weeks
• 🏢 EA Only Analysis: 13 IDGs across 3 weeks
• 🛍️ Jumbo.ae Analysis: 15 IDGs across 3 weeks
• 📅 Periods Analyzed: May 25, June 24, June 25
• 🗺️ Week Calculation: Based on Sunday as first day of month
• 📎 Export File: IDG_Weekly_Analysis_Combined.xlsx

🏆 All pivot tables created with comprehensive analysis!
📊 Ready for business insights and decision making!
